In [11]:
pip install numpy==1.23.4

Note: you may need to restart the kernel to use updated packages.


In [2894]:
import numpy as np
import pandas as pd
import cobra.test
import os

from os.path import join

from time import time


from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

import cobra
from cobra.test import create_test_model
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
model = create_test_model("textbook")
model.solver

a = cobra.io.read_sbml_model("iEC1344_C.xml")


In [2895]:
int_reactions= list()
i= -1
r = a.reactions[0]
r_id = r.id
zero= list()
one= list()
one_at_zero= list()

for r in a.reactions:  #finds reactions that have one feasible value at flux=0 or for another flux
    i= i+1
    r = a.reactions[i]
    r_id = r.id

    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    v= len(csvFile["Biomass"])-csvFile["Biomass"].isna().sum()

    flt= float(csvFile.iloc[100,  2])
    
    if v == 1 and csvFile.iloc[100,  2] != flt:
        one.append(r_id)
    elif v == 0:
        zero.append(r_id)
        
    if v == 1 and csvFile.iloc[100,  2] == flt:
        one_at_zero.append(r_id)

print(one)
print(zero)
print(one_at_zero)

D= pd.DataFrame (one, columns = ['1 feasible value'])
D_at_zero= pd.DataFrame (one_at_zero, columns = ['1 feasible value_at_zero'])


[]
[]
['EX_cm_e', 'EX_cobalt2_e', 'EX_colipa_e', 'EX_colipap_e', 'EX_crn_e', 'EX_crn__D_e', 'DM_4crsol_c', 'EX_cu2_e', 'DM_5drib_c', 'EX_cyan_e', 'DM_aacald_c', 'DM_amob_c', 'DM_mththf_c', 'EX_glyb_e', 'EX_gthox_e', 'BIOMASS_Ec_iJO1366_WT_53p95M', 'BIOMASS_Ec_iJO1366_core_53p95M', 'EX_14glucan_e', 'EX_15dap_e', 'EX_h2o2_e', 'EX_hacolipa_e', 'EX_halipa_e', 'EX_34dhpac_e', 'EX_hg2_e', 'EX_his__L_e', 'EX_4hoxpacd_e', 'EX_5mtr_e', 'EX_cys__D_e', 'EX_ile__L_e', 'EX_acgal_e', 'EX_inost_e', 'EX_k_e', 'EX_kdo2lipid4_e', 'EX_acolipa_e', 'EX_adocbl_e', 'EX_leu__L_e', 'EX_lipa_e', 'EX_lipa_cold_e', 'EX_ag_e', 'EX_lipoate_e', 'EX_ala_B_e', 'EX_malthx_e', 'EX_maltpt_e', 'EX_malttr_e', 'EX_maltttr_e', 'EX_arbt_e', 'EX_meoh_e', 'EX_met__D_e', 'EX_aso3_e', 'EX_met__L_e', 'EX_metsox_R__L_e', 'EX_metsox_S__L_e', 'EX_mg2_e', 'EX_mincyc_e', 'EX_btn_e', 'EX_mmet_e', 'EX_mn2_e', 'EX_ca2_e', 'EX_cbi_e', 'EX_mobd_e', 'EX_n2o_e', 'EX_na1_e', 'EX_nac_e', 'EX_cbl1_e', 'EX_cd2_e', 'EX_chol_e', 'EX_cl_e', 'EX_ni2_

In [2896]:
i= 0
r = a.reactions[0]
l2= list()
l3= list()

for r in a.reactions:            #separates reactions that have biomass values for positive and negative flux values
    
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    i3= 101

    while i3>100 and i3<len(csvFile):
        inf= csvFile.iloc[i3,  2]
        flt= float(csvFile.iloc[i3,  2])
    
        if csvFile.iloc[i3,  2] == flt and r_id not in l2:
            l2.append(r_id)     #l2 contains reactions that have biomass values for positive flux values
        i3= i3+1
    
    i4= 0
    
    while i4<100:
        inf= csvFile.iloc[i4,  2]
        flt= float(csvFile.iloc[i4,  2])
    
        if csvFile.iloc[i4,  2] == flt and r_id not in l3:
            l3.append(r_id)    #l3 contains reactions that have biomass values for negative flux values
        i4= i4+1

    i= i+1

D1= pd.DataFrame (l2, columns = ['Reaction names'])
D2= pd.DataFrame (l3, columns = ['Reaction names'])
b= D1.iloc[:, 0]
c= D2.iloc[:, 0]

union = pd.Series(np.union1d(b, c))
intersect = pd.Series(np.intersect1d(b, c))
bigb2= union[~union.isin(intersect)]

D3= pd.DataFrame (bigb2, columns = ['Reaction names'])
D3
D3.to_csv("res2 lactose.csv")      #contains reactions that have biomass values for either positive or negative flux values


In [2897]:
i= 0
indecr_and_stable_above_zero= list()
diff_indecr_above_zero= list()
flat_above_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(2)
    
    
    my_series = d['Biomass'].squeeze()


    if len(d)> 30 and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_above_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable_above_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_above_zero.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat_above_zero.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)
    
    i= i+1

flat_above_zero
diff_indecr_above_zero
indecr_and_stable_above_zero


['FRD3', 'O2tex', 'SPODM']

In [2898]:
i= 0
indecr_and_stable_below_zero= list()
diff_indecr_below_zero= list()
flat_below_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(2)
    
    my_series = d['Biomass'].squeeze()
    

    if len(d)> 30 and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_below_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 3 and count< len(d)-4:
            indecr_and_stable_below_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_below_zero.append(r_id)
        
        elif count> 2 and len(d) < 4:
            flat_below_zero.append(r_id)
        
        elif count> len(d)-4 and len(d)>4:
            flat_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)
    
    i= i+1

flat_below_zero
diff_indecr_below_zero
indecr_and_stable_below_zero


['EX_csn_e',
 'EX_glc__D_e',
 'EX_glcur_e',
 'EX_gln__L_e',
 'EX_glyc2p_e',
 'EX_h2_e',
 'EX_cynt_e',
 'EX_acgam1p_e',
 'EX_agm_e',
 'EX_alaala_e',
 'EX_man_e',
 'EX_manglyc_e',
 'EX_arg__L_e',
 'EX_o2_e',
 'EX_orn_e',
 'EX_ppa_e',
 'EX_ppal_e',
 'EX_ppt_e',
 'EX_psclys_e',
 'EX_r5p_e',
 'EX_fald_e',
 'EX_uacgam_e',
 'EX_udpg_e',
 'EX_udpgal_e',
 'EX_fru_e',
 'EX_frulys_e',
 'EX_fruur_e',
 'EX_g1p_e',
 'EX_gal_bD_e',
 'EX_gal1p_e',
 'ACOLIPAabctex',
 'EX_galctn__D_e',
 'EX_galctn__L_e',
 'EX_gam_e',
 'EX_gam6p_e',
 'CHLabcpp',
 'CLIPAabctex',
 'COLIPAabctex',
 'CU1abcpp',
 'DMSOR1',
 'DMSOR1pp',
 'DMSOR2',
 'DMSOR2pp',
 'ECA4COLIPAabctex',
 'DSBAO2',
 'ENLIPAabctex',
 'FUMt2_3pp',
 'ASPO3',
 'ASPO4',
 'ASPO5',
 'GALabcpp',
 'GLYBabcpp',
 'HYD2pp',
 'HYD3pp',
 'HG2abcpp',
 'LIPAabctex',
 'NO3R1bpp',
 'NO3R1pp',
 'NO3R2bpp',
 'NTRIR2x',
 'NTRIR3pp',
 'NTRIR4pp',
 'QMO2',
 'TMAOR1',
 'TMAOR1pp',
 'TMAOR2',
 'TMAOR2pp',
 'XYLabcpp',
 'PPPGO',
 'DKMPPD']

In [2899]:
i2= 0
r_id = indecr_and_stable_above_zero[0]
mon_incr_and_stable_above_zero= list()
mon_decr_and_stable_above_zero= list()

for r_id in indecr_and_stable_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_and_stable_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_above_zero)
set3= set(mon_incr_and_stable_above_zero)
inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_above_zero


['O2tex']

In [2900]:
w= list()
w2= list()
i2=0

while i2 < len(indecr_and_stable_above_zero):
    
    i=0
    r_id = indecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w2.append(r_id)

    
    i2= i2+1

w


['O2tex']

In [2901]:
set1= set(inANDdecr_and_stable_above_zero)
set2= set(w)
set3= set1-set2
inANDdecr_and_stable_above_zero= list(set3)

mon_decr_and_stable_above_zero= list(mon_decr_and_stable_above_zero) + list(w)


In [2902]:
i2= 0
r_id = indecr_and_stable_below_zero[0]
mon_incr_and_stable_below_zero= list()
mon_decr_and_stable_below_zero= list()

for r_id in indecr_and_stable_below_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_and_stable_below_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_and_stable_below_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_below_zero)
set2= set(mon_decr_and_stable_below_zero)
set3= set(mon_incr_and_stable_below_zero)
inANDdecr_and_stable_below_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero


['NTRIR4pp',
 'NO3R1bpp',
 'EX_gam_e',
 'NTRIR2x',
 'EX_h2_e',
 'EX_glc__D_e',
 'EX_o2_e',
 'EX_uacgam_e',
 'EX_manglyc_e',
 'EX_agm_e',
 'EX_cynt_e',
 'EX_udpgal_e',
 'EX_udpg_e',
 'NTRIR3pp',
 'EX_ppal_e',
 'EX_ppa_e',
 'EX_man_e',
 'PPPGO',
 'EX_fru_e']

In [2903]:
w3= list()
w4= list()
i2=0

while i2 < len(indecr_and_stable_below_zero):
    
    i=0
    r_id = indecr_and_stable_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3] < max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w3.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w4.append(r_id)

    
    i2= i2+1

w3


['NTRIR3pp', 'NTRIR4pp', 'PPPGO']

In [2904]:
w4

['EX_o2_e']

In [2905]:
inANDdecr_and_stable_below_zero.remove("NTRIR3pp")
inANDdecr_and_stable_below_zero.remove("NTRIR4pp")
inANDdecr_and_stable_below_zero.remove("PPPGO")
inANDdecr_and_stable_below_zero.remove("EX_o2_e")

set1= set(inANDdecr_and_stable_below_zero)
set2= set(w3)
set3= set(w4)
set4= set1-set2-set3
inANDdecr_and_stable_below_zero= list(set4)

mon_decr_and_stable_below_zero= list(mon_decr_and_stable_below_zero) + list(w3)
mon_incr_and_stable_below_zero= list(mon_incr_and_stable_below_zero) + list(w4)


In [2906]:
i= 0
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_both_positive_and_negative= list()
indecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
flat_below_zero_and_flat_above_zero= list()
diff_indecr_below_zero_and_flat_above_zero= list()
flat_below_zero_and_diff_indecr_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
indecr_and_stable_below_zero_and_diff_indecr_above_zero= list()
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_below_zero_and_indecr_and_stable_above_zero= list()
rest_both_positive_and_negative= list()
indecr_and_stable_below_zero_and_diff_indecr_above_zero2= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    count2=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))

    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n= 100
    l4= list()
    
    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l4.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()


    if (len(csvFile2) + len(csvFile3))> 30 and r_id in l3 and r_id in l2:
        
        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
        
        if (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> 0.05*len(csvFile3) or count2 == 0.05*len(csvFile3)) and count2< 0.95*len(csvFile3):
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)

        elif count> 0.95*len(csvFile2) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> 0.95*len(csvFile3)):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)

        elif count> 0.95*len(csvFile2) and count2> 0.95*len(csvFile3):
            flat_below_zero_and_flat_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and count2> 0.95*len(csvFile3):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
        
        elif count> 0.95*len(csvFile2) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2> 0.95*len(csvFile3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2< 0.05*len(csvFile3):
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            
        elif count< 0.05*len(csvFile2) and count2< 0.05*len(csvFile3):
            diff_indecr_both_positive_and_negative.append(r_id)

    
    if (len(csvFile2) + len(csvFile3))> 1 and (len(csvFile2) + len(csvFile3))<31  and r_id in l3 and r_id in l2:
        
        n2=0
        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
        
        if (count> 3 and count< len(csvFile2)-4) and (count2> 1 and len(csvFile3) < 3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> 3 and count< len(csvFile2)-4) and (count2> 3 and count2< len(csvFile3)-4):
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> 3 and count< len(csvFile2)-4) and (count2> 2 and len(csvFile3) < 4):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> 3 and count< len(csvFile2)-4) and (count2> len(csvFile3)-4 and len(csvFile3)>4):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> 1 and len(csvFile2) < 3) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> 2 and len(csvFile2) < 4) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-4 and len(csvFile2)>4) and (count2> 3 and count2< len(csvFile3)-4):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            
        elif (count< 4 and len(csvFile2)>4) and (count2> 3 and count2< len(csvFile3)-4):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            
        elif count> 3 and count2< 3:
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)

        else:
            rest_both_positive_and_negative.append(r_id)
    
    i= i+1

flat_below_zero_and_indecr_and_stable_above_zero
indecr_and_stable_below_zero_and_flat_above_zero
indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['EX_glcn_e',
 'EX_glyclt_e',
 'EX_h2o_e',
 'EX_hxan_e',
 'EX_cys__L_e',
 'EX_ac_e',
 'CBPS',
 'CYSabc2pp',
 'CYTBD2pp',
 'CYTBDpp',
 'ARGDCpp',
 'ARGabcpp',
 'F6PA',
 'FE2abcpp',
 'G3PD5',
 'G3PD6',
 'G3PD7',
 'GLCabcpp',
 'GLYC3Pabcpp',
 'H2Otex',
 'H2SO',
 'Kabcpp',
 'LDH_D2',
 'LYSabcpp',
 'MDH2',
 'MDH3',
 'MG2uabcpp',
 'NADH10',
 'NADH5',
 'NADH9',
 'NADPHQR2',
 'NADPHQR3',
 'NADPHQR4',
 'GTHPi',
 'NI2uabcpp',
 'ORNabcpp',
 'PTRCabcpp',
 'SULabcpp',
 'THIORDXi',
 'THRabcpp',
 'ZNabcpp',
 'POX']

In [2907]:
one_to_three_data_point_both_positive_and_negative= list()
rest2_both_positive_and_negative= list()
i=0

for r_id in rest_both_positive_and_negative:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    count2=0
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(rest_both_positive_and_negative[i]))

    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n= 100
    l4= list()
    
    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l4.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()


    if len(csvFile2) <4 and len(csvFile3)<4:
        one_to_three_data_point_both_positive_and_negative.append(rest_both_positive_and_negative[i])
    
    else:
        rest2_both_positive_and_negative.append(rest_both_positive_and_negative[i])

    i= i+1

one_to_three_data_point_both_positive_and_negative


['4PCP',
 '4PCPpp',
 '4PEPTabcpp',
 'ALAALAabcpp',
 'AGM3PA',
 'AGM3PApp',
 'AGM3PH',
 'AGM3Pt2pp',
 'AGM4PCP',
 'AM4PA',
 'AM4PCP',
 'AGM4PCPpp',
 'ETHAt2pp',
 'FACOAE80',
 'GNK',
 'MDDEP3pp',
 'MLDCP1Bpp',
 'MLDCP2Bpp',
 'MLDEP2pp',
 'MLTGY1pp',
 'MLTGY2pp']

In [2908]:
i=0
rest3_both_positive_and_negative= list()

for r_id in rest2_both_positive_and_negative:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat
    l= list()
    n=0
    n2=0
    count=0
    count2=0
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(rest2_both_positive_and_negative[i]))

    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n= 100
    l4= list()
    
    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l4.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    while n2 < len(csvFile2):

        if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
            count= count+1
    
        n2= n2+1
        
    n2=0
    while n2 < len(csvFile3):

        if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
            count2= count2+1
    
        n2= n2+1
        
    if count< 4 and count2< 4 and rest2_both_positive_and_negative not in diff_indecr_both_positive_and_negative:
        diff_indecr_both_positive_and_negative.append(rest2_both_positive_and_negative[i])
    elif count> 4 or count2> 4:
        rest3_both_positive_and_negative.append(rest2_both_positive_and_negative[i])
    
    i=i+1

diff_indecr_both_positive_and_negative


['ACALD',
 'CD2t3pp',
 'COBALT2t3pp',
 'CYStpp',
 'ARBt2rpp',
 'FBP',
 'ASNS2',
 'FE2tpp',
 'GLUDy',
 'GDPDPK',
 'GDPMNH',
 'GDPTPDP',
 'GLYK',
 'Kt2pp',
 'LYSt2pp',
 'MALt2_2pp',
 'MN2tpp',
 'NI2t3pp',
 'NI2tpp',
 'NMNAT',
 'PTRCt2pp',
 'PUNP1',
 'PUNP5',
 'PYK',
 'PYNP2r',
 'SUCASPtpp',
 'SUCCt2_2pp',
 'SUCMALtpp',
 'THD2pp',
 'THRt4pp',
 'TRE6PH',
 'XYLI2',
 'ZN2t3pp',
 'ZN2tpp',
 'PPCK',
 'PPK2',
 'EX_co2_e',
 'EX_val__L_e',
 '2AGPEAT120',
 '2AGPEAT140',
 '2AGPEAT141',
 '2AGPEAT160',
 '2AGPEAT161',
 '2AGPEAT180',
 '2AGPEAT181',
 '2AGPGAT120',
 '2AGPGAT140',
 '2AGPGAT141',
 '2AGPGAT160',
 '2AGPGAT161',
 '2AGPGAT180',
 '2AGPGAT181',
 'EX_ura_e',
 'AACPS5',
 'AACPS6',
 'AACPS7',
 'AACPS1',
 'AACPS2',
 'AACPS3',
 'AACPS4',
 'AKGDH',
 'AGM4PA',
 'CDAPPA181',
 'CLPNH120pp',
 'CLPNH140pp',
 'CLPNH141pp',
 'CLPNH160pp',
 'CLPNH161pp',
 'CLPNH180pp',
 'CLPNH181pp',
 'CBMKr',
 'CO2tex',
 'CO2tpp',
 'CDAPPA120',
 'CDAPPA140',
 'CDAPPA141',
 'CDAPPA160',
 'CDAPPA161',
 'CDAPPA180',
 'CPGNR1',


In [2909]:
rest3_both_positive_and_negative

[]

In [2910]:
i2= 0
r_id = indecr_and_stable_below_zero_and_indecr_and_stable_above_zero[0]
mon_incr_and_stable_both_positive_and_negative= list()
mon_decr_and_stable_both_positive_and_negative= list()

for r_id in indecr_and_stable_below_zero_and_indecr_and_stable_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)
    
    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_and_stable_both_positive_and_negative.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_and_stable_both_positive_and_negative.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_both_positive_and_negative)
set3= set(mon_incr_and_stable_both_positive_and_negative)
inANDdecr_and_stable_both_positive_and_negative= list(set1-set2-set3)
inANDdecr_and_stable_both_positive_and_negative


['Kabcpp',
 'ARGDCpp',
 'EX_h2o_e',
 'H2SO',
 'ZNabcpp',
 'H2Otex',
 'CBPS',
 'NI2uabcpp',
 'F6PA',
 'EX_ac_e',
 'SULabcpp']

In [2911]:
i2= 0
r_id = diff_indecr_below_zero_and_flat_above_zero[0]
mon_incr_below_zero_and_flat_above_zero= list()
mon_decr_below_zero_and_flat_above_zero= list()

for r_id in diff_indecr_below_zero_and_flat_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_below_zero_and_flat_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr_below_zero_and_flat_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr_below_zero_and_flat_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(diff_indecr_below_zero_and_flat_above_zero)
set2= set(mon_decr_below_zero_and_flat_above_zero)
set3= set(mon_incr_below_zero_and_flat_above_zero)
inANDdecr_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_flat_above_zero


['PTAr']

In [2912]:
mon_incr_below_zero_and_flat_above_zero

['CRNDt2rpp', 'DURAD']

In [2913]:
mon_decr_below_zero_and_flat_above_zero

[]

In [2914]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_below_zero_and_flat_above_zero):
    
    i=0
    r_id = inANDdecr_below_zero_and_flat_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    i2= i2+1
    
w5


[]

In [2915]:
w6

set1= set(inANDdecr_below_zero_and_flat_above_zero)
set2= set(w6)
set3= set1-set2
inANDdecr_below_zero_and_flat_above_zero= list(set3)

mon_incr_below_zero_and_flat_above_zero= list(mon_incr_below_zero_and_flat_above_zero) + list(w6)


In [2916]:
i2= 0
r_id = flat_below_zero_and_diff_indecr_above_zero[0]
flat_below_zero_and_mon_incr_above_zero= list()
flat_below_zero_and_mon_decr_above_zero= list()

for r_id in flat_below_zero_and_diff_indecr_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=100
    r_id = flat_below_zero_and_diff_indecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            flat_below_zero_and_mon_decr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            flat_below_zero_and_mon_incr_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(flat_below_zero_and_diff_indecr_above_zero)
set2= set(flat_below_zero_and_mon_decr_above_zero)
set3= set(flat_below_zero_and_mon_incr_above_zero)
flat_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_above_zero


[]

In [2917]:
flat_below_zero_and_mon_decr_above_zero

['ACKr',
 'ALATA_D2',
 'ALATA_L2',
 'ADNK1',
 'ADNUC',
 'ADPT',
 'AMPN',
 'ASPt2pp_copy2',
 'F6PP',
 'FA120ACPHi',
 'FA140ACPHi',
 'FA141ACPHi',
 'FA160ACPHi',
 'FA161ACPHi',
 'ASNN',
 'FACOAL60t2pp',
 'ASNNpp',
 'FTHFLi',
 'G6PP',
 'ASPT',
 'GLUN',
 'GART',
 'GDPMNP',
 'GLUSy',
 'HXCT',
 'HXPRT',
 'IDOND2',
 'HEX1',
 'HEX7',
 'INSH',
 'INSK',
 'HPYRRy',
 'MLTG3',
 'MLTG4',
 'MLTG5',
 'MN6PP',
 'GSNK',
 'NADTRHD',
 'NTD7',
 'NTD9',
 'NTPTP2',
 'PFK',
 'PFK_3',
 'GUAPRT',
 'PDH',
 'PRPPS',
 'R5PP',
 'THRA2',
 'UPPRT',
 'URIH',
 'URIK2',
 'PPK',
 'PPM',
 'PYRDC']

In [2918]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(flat_below_zero_and_inANDdecr_above_zero):
    
    i=0
    r_id = flat_below_zero_and_inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    i2= i2+1
    
w5


[]

In [2919]:
i2= 0
r_id = indecr_and_stable_below_zero_and_diff_indecr_above_zero[0]
indecr_and_stable_below_zero_and_mon_incr_above_zero= list()
indecr_and_stable_below_zero_and_mon_decr_above_zero= list()

for r_id in indecr_and_stable_below_zero_and_diff_indecr_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    n2= 100
    r_id = indecr_and_stable_below_zero_and_diff_indecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            indecr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            indecr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_diff_indecr_above_zero)
set2= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(indecr_and_stable_below_zero_and_mon_incr_above_zero)
indecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
indecr_and_stable_below_zero_and_inANDdecr_above_zero


['ATPS4rpp', 'FUM', 'MDH']

In [2920]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(indecr_and_stable_below_zero_and_inANDdecr_above_zero):
    
    i=0
    r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 101:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 100
    l= list()
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1
    
w5


['ATPS4rpp', 'FUM']

In [2921]:
w7

['FUM', 'MDH']

In [2922]:
not_truly_indecr_and_stable_below_zero_and_inANDdecr_above_zero= list(w5)+list(w6)+list(w7)
not_truly_indecr_and_stable_below_zero_and_inANDdecr_above_zero


['ATPS4rpp', 'FUM', 'FUM', 'MDH']

In [2923]:
i2= 0
r_id = flat_below_zero_and_indecr_and_stable_above_zero[0]
flat_below_zero_and_incr_and_stable_above_zero= list()
flat_below_zero_and_decr_and_stable_above_zero= list()

for r_id in flat_below_zero_and_indecr_and_stable_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=100
    r_id = flat_below_zero_and_indecr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            flat_below_zero_and_decr_and_stable_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            flat_below_zero_and_incr_and_stable_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(flat_below_zero_and_indecr_and_stable_above_zero)
set2= set(flat_below_zero_and_decr_and_stable_above_zero)
set3= set(flat_below_zero_and_incr_and_stable_above_zero)
flat_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [2924]:
flat_below_zero_and_decr_and_stable_above_zero

['EX_h_e',
 'EX_fe2_e',
 'ACACT8r_copy1',
 'ADA',
 'ADD',
 'DADA',
 'CTECOAI6_copy1',
 'CTECOAI7_copy1',
 'CTECOAI8_copy1',
 'DHAPT',
 'DAAD',
 'DHORD2',
 'DHORDfum',
 'EAR100y',
 'EAR120y',
 'EAR121y',
 'EAR140y',
 'EAR141y',
 'EAR160y',
 'EAR161y',
 'EAR180y',
 'EAR181y',
 'FADRx2',
 'FDH4pp',
 'ASNS1',
 'FEROpp',
 'FLVR',
 'FMNRx2',
 'FE3tex',
 'GLUt2rpp',
 'GLYCK',
 'GLYCK2',
 'GLYCLTDy',
 'GLYCTO2',
 'GLYOX3',
 'L_LACD2_copy1',
 'L_LACD3_copy1',
 'MDDCP3pp',
 'ME1',
 'MLDCP2App',
 'MMM',
 'MICITDr_copy1',
 'NADH16pp',
 'NTD8',
 'NTPTP1',
 'GTPDPDP',
 'OAADC',
 'SSALx',
 'SUCOAS',
 'SERD_D',
 'SERD_L',
 'THRA',
 'THRD',
 'TRPS2',
 'TRSARr',
 'XTSNH_copy1',
 'L_LACD3_copy2',
 'MMM2',
 'L_LACD2_copy2',
 'XTSNH_copy2']

In [2925]:
i2= 0
r_id = indecr_and_stable_below_zero_and_flat_above_zero[0]
incr_and_stable_below_zero_and_flat_above_zero= list()
decr_and_stable_below_zero_and_flat_above_zero= list()

for r_id in indecr_and_stable_below_zero_and_flat_above_zero:                    
    
    i=0
    r_id = indecr_and_stable_below_zero_and_flat_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_and_stable_below_zero_and_flat_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_and_stable_below_zero_and_flat_above_zero.append(r_id)

    
    i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_flat_above_zero)
set2= set(incr_and_stable_below_zero_and_flat_above_zero)
set3= set(decr_and_stable_below_zero_and_flat_above_zero)
inANDdecr_and_stable_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_flat_above_zero


['Htex', 'EX_fe3_e']

In [2926]:
incr_and_stable_below_zero_and_flat_above_zero

['ACACT4r',
 'ACACT5r',
 'ACACT6r',
 'ACACT7r',
 'FE2tex',
 'HACD4',
 'HACD5',
 'HACD6',
 'HACD7',
 'HACD8']

In [2927]:
decr_and_stable_below_zero_and_flat_above_zero

[]

In [2928]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_and_stable_below_zero_and_flat_above_zero):
    
    i=0
    r_id = inANDdecr_and_stable_below_zero_and_flat_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 101:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 100
    l= list()
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1
    
w5


[]

In [2929]:
w8

incr_and_stable_below_zero_and_flat_above_zero= list(incr_and_stable_below_zero_and_flat_above_zero)+list(w8)


In [2930]:
i2= 0
r_id = diff_indecr_both_positive_and_negative[0]
incr_both_positive_and_negative= list()
decr_both_positive_and_negative= list()

for r_id in diff_indecr_both_positive_and_negative:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_both_positive_and_negative[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_both_positive_and_negative.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_both_positive_and_negative.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point_both_positive_and_negative.append(r_id)

    
    i2= i2 + 1


set1= set(diff_indecr_both_positive_and_negative)
set2= set(decr_both_positive_and_negative)
set3= set(incr_both_positive_and_negative)
inANDdecr_both_positive_and_negative= list(set1-set2-set3)
inANDdecr_both_positive_and_negative


['FACOAE120',
 'GDPTPDP',
 'EX_val__L_e',
 '2AGPGAT120',
 '2AGPGAT180',
 'CPGNR2',
 'ACALD',
 'PGM',
 'FACOAE160',
 'MGSA',
 'TKT2',
 'NMNN',
 'ASNS2',
 'CLPNH180pp',
 'FE3HOXR1',
 '2AGPEAT180',
 'PPC',
 'PTRCt2pp',
 'DURIPP',
 'AACPS4',
 'CDAPPA141',
 'FEOXAMR2',
 'PGPP180',
 'CPGNR3',
 'GAPD',
 'NI2t3pp',
 'AACPS3',
 'PGPP140',
 'RPE',
 'PUNP2',
 'SUCASPtpp',
 'SUCDi',
 'PGPP160',
 'LPLIPAL2E140',
 'FACOAE181',
 'THD2pp',
 'G3PGabcpp',
 'MTHFD',
 'ORNDC',
 'GTHRDabcpp',
 'FACOAE140',
 'PA120abcpp',
 'RNDR3b',
 'FE2tpp',
 '2AGPEAT160',
 'CLPNH120pp',
 'CYSDS',
 'CO2tpp',
 'GLYK',
 'THRt4pp',
 'FA100ACPHi',
 'GLUDy',
 '2AGPGAT181',
 'RNDR3',
 'ZN2tpp',
 'CDAPPA160',
 'PPK2',
 '2AGPEAT161',
 'GHMT2r',
 'RNDR1b',
 'FEOXAMR3',
 'NACODA',
 'LPLIPAL2G120',
 'TMDK1',
 'FECRMR1',
 'FACOAE141',
 'ENO',
 'PA160abcpp',
 '2AGPGAT140',
 'RNTR1c',
 'P5CD',
 '2AGPGAT160',
 'GDPDPK',
 'UM3PL',
 'LPLIPAL2G161',
 'PROD2',
 'CLPNH141pp',
 '2AGPGAT161',
 'MN2tpp',
 'LYSt2pp',
 'TPI',
 'HXAND',
 'ICL',
 '

In [2931]:
i=0
A= list()
b= list()
c= list()
d= list()
e= list()
decr_below_zero_and_decr_and_stable_above_zero=list()
decr_below_zero_and_incr_and_stable_above_zero= list()
incr_below_zero_and_incr_and_stable_above_zero= list()
incr_below_zero_and_decr_and_stable_above_zero= list()
inANDdecr_below_zero_and_decr_and_stable_above_zero= list()
inANDdecr_below_zero_and_incr_and_stable_above_zero= list()
inANDdecr_below_zero_and_decr_and_stable_above_zero=list()
inANDdecr_below_zero_and_incr_and_stable_above_zero=list()
decr_below_zero_and_inANDdecr_and_stable_above_zero=list()
incr_below_zero_and_inANDdecr_and_stable_above_zero=list()
rest= list()
rest2= list()

while i < len(diff_indecr_below_zero_and_indecr_and_stable_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if len(csvFile2['Biomass'])>3:
        
        if csvFile2["Biomass"].is_monotonic_decreasing ==True and csvFile3["Biomass"].is_monotonic_decreasing ==True and r_id not in decr_below_zero_and_decr_and_stable_above_zero:
            decr_below_zero_and_decr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic_decreasing ==True and csvFile3["Biomass"].is_monotonic ==True and r_id not in decr_below_zero_and_incr_and_stable_above_zero:
            decr_below_zero_and_incr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic ==True and csvFile3["Biomass"].is_monotonic ==True and r_id not in incr_below_zero_and_incr_and_stable_above_zero:
            incr_below_zero_and_incr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic ==True and csvFile3["Biomass"].is_monotonic_decreasing ==True and r_id not in incr_below_zero_and_decr_and_stable_above_zero:
            incr_below_zero_and_decr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic ==False and csvFile2["Biomass"].is_monotonic_decreasing ==False and csvFile3["Biomass"].is_monotonic_decreasing ==True and r_id not in inANDdecr_below_zero_and_decr_and_stable_above_zero:
            inANDdecr_below_zero_and_decr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic ==False and csvFile2["Biomass"].is_monotonic_decreasing ==False and csvFile3["Biomass"].is_monotonic ==True and r_id not in inANDdecr_below_zero_and_incr_and_stable_above_zero:
            inANDdecr_below_zero_and_incr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic_decreasing ==True and csvFile3["Biomass"].is_monotonic ==False and csvFile3["Biomass"].is_monotonic_decreasing ==False and r_id not in decr_below_zero_and_inANDdecr_and_stable_above_zero:
            decr_below_zero_and_inANDdecr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
        elif csvFile2["Biomass"].is_monotonic ==True and csvFile3["Biomass"].is_monotonic ==False and csvFile3["Biomass"].is_monotonic_decreasing ==False and r_id not in incr_below_zero_and_inANDdecr_and_stable_above_zero:
            incr_below_zero_and_inANDdecr_and_stable_above_zero.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])

    else:
        rest2.append(diff_indecr_below_zero_and_indecr_and_stable_above_zero[i])
    
    i=i+1
    
incr_below_zero_and_decr_and_stable_above_zero


['CMPN',
 'CYTD',
 'ARGDC',
 'EAR100x',
 'EAR120x',
 'EAR121x',
 'EAR140x',
 'EAR141x',
 'EAR160x',
 'EAR161x',
 'EAR180x',
 'EAR181x',
 'FADRx',
 'FBA',
 'FLVRx',
 'FMNRx',
 'FRD2',
 'GLCt2pp',
 'GLYC3Pt6pp',
 'GLYCLTDx',
 'IDOND',
 'IMPD',
 'LALDO2x',
 'ME2',
 'MOX',
 'NADH17pp',
 'NADH18pp',
 'PGI',
 'PIt2rpp',
 'RNTR1c2',
 'RNTR2c2',
 'RNTR3c2',
 'RNTR4c2',
 'SSALy',
 'SO4t2pp',
 'POR5']

In [2932]:
decr_below_zero_and_decr_and_stable_above_zero

['EX_glu__L_e',
 'EX_gly_e',
 'EX_glyald_e',
 'EX_glyc3p_e',
 'EX_5dglcn_e',
 'EX_acald_e',
 'EX_lac__D_e',
 'EX_lac__L_e',
 'EX_akg_e',
 'EX_ala__L_e',
 'EX_mal__L_e',
 'EX_asp__L_e',
 'EX_dha_e',
 'EX_pyr_e',
 'EX_ser__L_e',
 'EX_etha_e',
 'EX_thr__L_e']

In [2933]:
incr_below_zero_and_incr_and_stable_above_zero

['GLCptspp']

In [2934]:
incr_below_zero_and_inANDdecr_and_stable_above_zero

['CYTBO3_4pp']

In [2935]:
inANDdecr_below_zero_and_decr_and_stable_above_zero

['EX_glyc_e',
 'EX_glyc__R_e',
 'EX_gua_e',
 'EX_12ppd__S_e',
 'EX_4abut_e',
 'EX_idon__L_e',
 'EX_ade_e',
 'EX_ala__D_e',
 'EX_alltn_e',
 'EX_asn__L_e',
 'EX_xan_e',
 'EX_pro__L_e',
 'EX_succ_e',
 'EX_etoh_e',
 'EX_for_e',
 'CAT',
 'FE3abcpp',
 'GLYCDx',
 'H2Otpp']

In [2936]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_below_zero_and_decr_and_stable_above_zero):
    
    i=0
    r_id = inANDdecr_below_zero_and_decr_and_stable_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 101:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 100
    l= list()
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


['EX_ala__D_e', 'EX_succ_e', 'EX_for_e']

In [2937]:
w6

['GLYCDx']

In [2938]:
set1= set(inANDdecr_below_zero_and_decr_and_stable_above_zero)
set2= set(w5)
set3= set1-set2
inANDdecr_below_zero_and_decr_and_stable_above_zero= list(set3)

decr_below_zero_and_decr_and_stable_above_zero= list(decr_below_zero_and_decr_and_stable_above_zero) + list(w5)


In [2939]:
set1= set(inANDdecr_below_zero_and_decr_and_stable_above_zero)
set2= set(w6)
set3= set1-set2
inANDdecr_below_zero_and_decr_and_stable_above_zero= list(set3)

incr_below_zero_and_decr_and_stable_above_zero= list(incr_below_zero_and_decr_and_stable_above_zero) + list(w6)


In [2940]:
i=0
A= list()
b= list()
c= list()
d= list()
e= list()
decreasing_and_stable_below_zero_and_decreasing_above_zero= list()
decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero= list()
flat_below_zero_and_decreasing_and_stable_above_zero= list()
decreasing_and_stable_below_zero_and_flat_above_zero= list()


while i < len(mon_decr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(mon_decr_and_stable_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        A.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) > 1:
            decreasing_and_stable_below_zero_and_decreasing_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) == 1:
            flat_below_zero_and_mon_decr_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
    
    if boolean == False and boolean2 == False:
        b.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean == False and boolean2 == True:
        c.append(mon_decr_and_stable_both_positive_and_negative[i])

    if boolean == True:
        d.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean2 == True:
        e.append(mon_decr_and_stable_both_positive_and_negative[i])
    
    if boolean == True and boolean2 == True:
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) > 1:
            decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) == 1 and len(csvFile3['Biomass'].unique()) > 1:
            flat_below_zero_and_decreasing_and_stable_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) == 1:
            decreasing_and_stable_below_zero_and_flat_above_zero.append(mon_decr_and_stable_both_positive_and_negative[i])

    i=i+1

decreasing_and_stable_below_zero_and_decreasing_above_zero


['EX_glcn_e',
 'EX_glyclt_e',
 'EX_hxan_e',
 'EX_cys__L_e',
 'CYSabc2pp',
 'CYTBD2pp',
 'CYTBDpp',
 'ARGabcpp',
 'FE2abcpp',
 'G3PD5',
 'G3PD6',
 'G3PD7',
 'GLCabcpp',
 'GLYC3Pabcpp',
 'LDH_D2',
 'LYSabcpp',
 'MDH2',
 'MDH3',
 'MG2uabcpp',
 'NADH10',
 'NADH5',
 'NADH9',
 'NADPHQR2',
 'NADPHQR3',
 'NADPHQR4',
 'GTHPi',
 'ORNabcpp',
 'PTRCabcpp',
 'THIORDXi',
 'THRabcpp',
 'POX']

In [2941]:
flat_below_zero_and_mon_decr_above_zero

['ACKr',
 'ALATA_D2',
 'ALATA_L2',
 'ADNK1',
 'ADNUC',
 'ADPT',
 'AMPN',
 'ASPt2pp_copy2',
 'F6PP',
 'FA120ACPHi',
 'FA140ACPHi',
 'FA141ACPHi',
 'FA160ACPHi',
 'FA161ACPHi',
 'ASNN',
 'FACOAL60t2pp',
 'ASNNpp',
 'FTHFLi',
 'G6PP',
 'ASPT',
 'GLUN',
 'GART',
 'GDPMNP',
 'GLUSy',
 'HXCT',
 'HXPRT',
 'IDOND2',
 'HEX1',
 'HEX7',
 'INSH',
 'INSK',
 'HPYRRy',
 'MLTG3',
 'MLTG4',
 'MLTG5',
 'MN6PP',
 'GSNK',
 'NADTRHD',
 'NTD7',
 'NTD9',
 'NTPTP2',
 'PFK',
 'PFK_3',
 'GUAPRT',
 'PDH',
 'PRPPS',
 'R5PP',
 'THRA2',
 'UPPRT',
 'URIH',
 'URIK2',
 'PPK',
 'PPM',
 'PYRDC']

In [2942]:
z=0
f= list()
g= list()
h= list()
i= list()
j= list()
flat_below_zero_and_increasing_above_zero= list()
increasing_and_stable_below_zero_and_decreasing_above_zero= list()
increasing_and_stable_below_zero_and_increasing_and_stable_above_zero= list()
increasing_below_zero_and_flat_above_zero= list()
flat_below_zero_and_increasing_and_stable_above_zero= list()
mon_incr_and_stable_below_zero_and_flat_above_zero= list()

while z < len(mon_incr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(mon_incr_and_stable_both_positive_and_negative[z]))
    
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1
    
    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        f.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 1:
            increasing_and_stable_below_zero_and_decreasing_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) == 1:
            flat_below_zero_and_increasing_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
    
    if boolean == False and boolean2 == False:
        g.append(mon_incr_and_stable_both_positive_and_negative[z])
    
    if boolean == False and boolean2 == True:
        h.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile3['Biomass'].unique()) == 1:
            
            increasing_below_zero_and_flat_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])

    if boolean == True:
        i.append(mon_incr_and_stable_both_positive_and_negative[z])
    
    if boolean2 == True:
        j.append(mon_incr_and_stable_both_positive_and_negative[z])
    
    if boolean == True and boolean2 == True:
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) > 1:
            increasing_and_stable_below_zero_and_increasing_and_stable_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) == 1 and len(csvFile3['Biomass'].unique()) > 1:
            flat_below_zero_and_increasing_and_stable_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) == 1:
            mon_incr_and_stable_below_zero_and_flat_above_zero.append(mon_incr_and_stable_both_positive_and_negative[z])

    z=z+1

increasing_and_stable_below_zero_and_decreasing_above_zero


[]

In [2943]:
z=0
k= list()
L= list()
m= list()
o= list()
p= list()
increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero= list()
flat_below_zero_and_increasingANDdecreasing_above_zero= list()
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list()
decreasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list()
increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_increasing_and_stable_above_zero= list()
flat_below_zero_and_inANDdecr_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_flat_above_zero= list()
decreasing_and_stable_below_zero_and_increasing_and_stable_above_zero= list()

while z < len(inANDdecr_and_stable_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(inANDdecr_and_stable_both_positive_and_negative[z]))
    
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    boolean = csvFile2["Biomass"].duplicated().any()
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    boolean2 = csvFile3["Biomass"].duplicated().any()
    
    if boolean == True and boolean2 == False:
        k.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 1:
            increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) == 1:
            flat_below_zero_and_increasingANDdecreasing_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
    
    if boolean == False and boolean2 == False:
        L.append(inANDdecr_and_stable_both_positive_and_negative[z])
    
    if boolean == False and boolean2 == True:
        m.append(inANDdecr_and_stable_both_positive_and_negative[z])

    if boolean == True:
        o.append(inANDdecr_and_stable_both_positive_and_negative[z])
    
    if boolean2 == True:
        p.append(inANDdecr_and_stable_both_positive_and_negative[z])
    
    if boolean == True and boolean2 == True:
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) > 1:
            inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
                
        if len(csvFile2['Biomass'].unique()) == 1 and len(csvFile3['Biomass'].unique()) > 1:
            flat_below_zero_and_inANDdecr_and_stable_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])
        
        if len(csvFile2['Biomass'].unique()) > 1 and len(csvFile3['Biomass'].unique()) == 1:
            inANDdecr_and_stable_below_zero_and_flat_above_zero.append(inANDdecr_and_stable_both_positive_and_negative[z])

        
    z=z+1

increasingORANDdecreasing_and_stable_below_zero_and_increasingORANDdecreasing_above_zero


['Kabcpp',
 'ARGDCpp',
 'H2SO',
 'ZNabcpp',
 'H2Otex',
 'CBPS',
 'NI2uabcpp',
 'EX_ac_e',
 'SULabcpp']

In [2944]:
L

increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero= list(L)
increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero


['F6PA']

In [2945]:
m

incr_below_zero_and_inANDdecr_and_stable_above_zero.append("EX_h2o_e")


In [2946]:
inANDdecr_and_stable_below_zero_and_decreasing_above_zero= ["H2Otex", "EX_ac_e"]
inANDdecr_and_stable_below_zero_and_decreasing_above_zero


['H2Otex', 'EX_ac_e']

In [2947]:
decreasing_and_stable_below_zero_and_decreasing_above_zero.append("NI2uabcpp")
decreasing_and_stable_below_zero_and_decreasing_above_zero.append("SULabcpp")
decreasing_and_stable_below_zero_and_decreasing_above_zero.append("CBPS")
decreasing_and_stable_below_zero_and_decreasing_above_zero.append("ZNabcpp")
decreasing_and_stable_below_zero_and_decreasing_above_zero.append("H2SO")
decreasing_and_stable_below_zero_and_decreasing_above_zero.append("Kabcpp")
decreasing_and_stable_below_zero_and_decreasing_above_zero.append("ARGDCpp")


In [2948]:
i2= 0
r_id = indecr_and_stable_below_zero_and_mon_decr_above_zero[0]
incr_and_stable_below_zero_and_mon_decr_above_zero= list()
decr_and_stable_below_zero_and_mon_decr_above_zero= list()
one_to_three_data_point_below_zero_and_mon_decr_above_zero= list()

for r_id in indecr_and_stable_below_zero_and_mon_decr_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 101:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point_below_zero_and_mon_decr_above_zero.append(r_id)

    
    i2= i2 + 1

decr_and_stable_below_zero_and_mon_decr_above_zero

set1= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set2= set(decr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(incr_and_stable_below_zero_and_mon_decr_above_zero)
set4= set(one_to_three_data_point_below_zero_and_mon_decr_above_zero)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero= list(set1-set2-set3-set4)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero


['MNt2pp', 'PIuabcpp', 'FLDR2', 'PPA']

In [2949]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero):
    
    i=0
    r_id = inANDdecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 101:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 100
    l= list()
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


['MNt2pp', 'PIuabcpp', 'PPA']

In [2950]:
w6

set1= set(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)
set2= set(w5)
set3= set1-set2
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero= list(set3)

set1= set(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)
set2= set(w6)
set3= set1-set2
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero= list(set3)

decr_and_stable_below_zero_and_mon_decr_above_zero= list(decr_and_stable_below_zero_and_mon_decr_above_zero)+list(w5)
incr_and_stable_below_zero_and_mon_decr_above_zero= list(incr_and_stable_below_zero_and_mon_decr_above_zero)+list(w6)


In [2951]:
i2= 0
r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[0]
incr_and_stable_below_zero_and_inANDdecr_above_zero= list()
decr_and_stable_below_zero_and_inANDdecr_above_zero= list()
one_to_three_data_point_below_zero_and_inANDdecr_above_zero= list()

for r_id in indecr_and_stable_below_zero_and_inANDdecr_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 101:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_and_stable_below_zero_and_inANDdecr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_and_stable_below_zero_and_inANDdecr_above_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point_below_zero_and_inANDdecr_above_zero.append(r_id)

    
    i2= i2 + 1

decr_and_stable_below_zero_and_inANDdecr_above_zero

set1= set(indecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(decr_and_stable_below_zero_and_inANDdecr_above_zero)
set3= set(incr_and_stable_below_zero_and_inANDdecr_above_zero)
set4= set(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3-set4)
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero


[]

In [2952]:
incr_and_stable_below_zero_and_inANDdecr_above_zero

['ATPS4rpp', 'MDH']

In [2953]:
one_to_three_data_point_below_zero_and_inANDdecr_above_zero

['FUM']

In [2954]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(incr_and_stable_below_zero_and_inANDdecr_above_zero):
    
    i=0
    r_id = incr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 101:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 100
    l= list()
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


['ATPS4rpp']

In [2955]:
w7

set1= set(incr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(w7)
set3= set1-set2
incr_and_stable_below_zero_and_inANDdecr_above_zero= list(set3)

incr_and_stable_below_zero_and_mon_decr_above_zero= list(incr_and_stable_below_zero_and_mon_decr_above_zero)+w7


In [2956]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero):
    
    i=0
    r_id = one_to_three_data_point_below_zero_and_inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 101:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 100
    l= list()
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


['FUM']

In [2957]:
w7

one_to_three_data_point_below_zero_and_mon_decr_above_zero.append("FUM")
one_to_three_data_point_below_zero_and_mon_decr_above_zero


['FUM']

In [2958]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero):
    
    i=0
    r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 101:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 100
    l= list()
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


[]

In [2959]:
w6

[]

In [2960]:
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero.remove("PPA")
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero.remove("PIuabcpp")
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero.remove("MNt2pp")
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero.remove("FLDR2")
decr_and_stable_below_zero_and_inANDdecr_above_zero.append("PPA")
decr_and_stable_below_zero_and_inANDdecr_above_zero.append("PIuabcpp")
decr_and_stable_below_zero_and_inANDdecr_above_zero.append("MNt2pp")
incr_and_stable_below_zero_and_inANDdecr_above_zero.append("FLDR2")


ValueError: list.remove(x): x not in list

In [2961]:
i2= 0
r_id = diff_indecr_above_zero[0]
incr_above_zero= list()
decr_above_zero= list()
one_to_three_data_point_above_zero= list()

for r_id in diff_indecr_above_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_above_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point_above_zero.append(r_id)

    
    i2= i2 + 1

decr_above_zero


['2AGPA120tipp',
 '2AGPA140tipp',
 '2AGPA141tipp',
 '2AGPA160tipp',
 '2AGPA161tipp',
 '2AGPA180tipp',
 '2AGPA181tipp',
 '2AGPE120tipp',
 '2AGPE140tipp',
 '2AGPE141tipp',
 '2AGPE160tipp',
 '2AGPE161tipp',
 '2AGPE180tipp',
 '2AGPE181tipp',
 '2AGPG120tipp',
 '2AGPG140tipp',
 '2AGPG141tipp',
 '2AGPG160tipp',
 '2AGPG161tipp',
 '2AGPG180tipp',
 '2AGPG181tipp',
 'EX_urea_e',
 '2MAHMP',
 '3HAD120',
 '3HAD140',
 '3HAD160',
 '3HAD161',
 'AACPS8',
 '3HAD180',
 '3HAD181',
 'AACPS9',
 'AACTOOR',
 'ABTA',
 'ABUTD',
 '3OAR120',
 '3OAR160',
 '3OAR161',
 '3OAR180',
 '3OAR181',
 '3OAS120',
 '3OAS160',
 '3OAS161',
 '3OAS180',
 '3OAS181',
 'ACOAD5f',
 'ACOAD6f',
 'ACOAD7f',
 'ACOAD8f',
 'ACPPAT120',
 'ACPPAT140',
 'ACPPAT141',
 'AGPAT181',
 'ACPPAT180',
 'ACPPAT181',
 'ADNCYC',
 'ADPRDP',
 'AGDC',
 'AGMT',
 'AGPAT120',
 'AGPAT140',
 'AOBUTDs',
 'AGPAT141',
 'AMPTASECG',
 'AGPAT180',
 'AP4AH',
 'AP4AS',
 'APG3PAT120',
 'APG3PAT140',
 'APG3PAT141',
 'AST',
 'ATPHs',
 'APG3PAT180',
 'CGLYabcpp',
 'APG3PAT181

In [2962]:
one_to_three_data_point_above_zero

['DM_oxam_c',
 'EX_12ppd__R_e',
 'EX_h2s_e',
 'EX_3hpp_e',
 'EX_hom__L_e',
 'EX_acser_e',
 'EX_cit_e',
 'EX_thym_e',
 '3HAD100',
 '3AMACHYD',
 '3HAD121',
 '3HAD141',
 '3HPPtpp',
 '3OAR100',
 '3OAR121',
 '3OAR141',
 '3OAS100',
 '3OAS121',
 '3OAS141',
 'ACGAMK',
 '3PEPTabcpp',
 'ACM6PH',
 'ACOAD4f',
 'AICART',
 'AIRC2',
 'ACSERtpp',
 'ALAGLUE',
 'ALLTAMH',
 'ALLTN',
 'ADSK',
 'ADSL2r',
 'AGM4PApp',
 'ALR2',
 'ALR4x',
 'AM3PA',
 'AGM4PH',
 'AGM4Pt2pp',
 'AGMH',
 'AGMt2pp',
 'ANHMK',
 'BPNT',
 'CITt3pp',
 'CBMD',
 'DHFR',
 'G1SAT',
 'ASPCT',
 'GLCDpp',
 'GLCNt2rpp',
 'GLUPRT',
 'GLUTRR',
 'GLUTRS',
 'H2St1pp',
 'HOMt2pp',
 'LPLIPAL1E181pp',
 'LADGMDH',
 'LALGP',
 'LCARR',
 'LCARS',
 'LPLIPAL1E120pp',
 'LPLIPAL1E140pp',
 'LPLIPAL1E141pp',
 'LPLIPAL1E160pp',
 'LPLIPAL1E161pp',
 'LPLIPAL1E180pp',
 'MCTP1App',
 'MCTP1Bpp',
 'MDDCP1pp',
 'MDDCP4pp',
 'MDDCP5pp',
 'MDDEP1pp',
 'MDDEP2pp',
 'MLDCP1App',
 'MLDCP3App',
 'MLDEP1pp',
 'GPDDA2pp',
 'MLTGY3pp',
 'MPTG',
 'MSAR',
 'OMPDC',
 'OXAMTC',
 '

In [2963]:
incr_above_zero

[]

In [2964]:
set1= set(diff_indecr_above_zero)
set2= set(decr_above_zero)
set3= set(incr_above_zero)
set4= set(one_to_three_data_point_above_zero)
inANDdecr_above_zero= list(set1-set2-set3-set4)
inANDdecr_above_zero


['APG3PAT160',
 'ACGS',
 'DTMPK',
 'ACODA',
 'APG3PAT161',
 '3OAS140',
 'HCO3E',
 'NADK',
 'NH4tpp',
 'CTPS2',
 'CS',
 'ACCOAC',
 'ACPPAT161',
 'O2tpp',
 'G5SADs',
 'AGPAT160',
 'GF6PTA',
 'GK1',
 'PSSA161',
 'ALAALAr',
 'ICDHyr',
 'NDPK4',
 'ASPK',
 'UAGDP',
 'PSSA160',
 'PE160abcpp',
 '3OAR140',
 'DASYN160',
 'CYSS',
 'ACONTb',
 'CU2tpp',
 'MCOATA',
 'NNATr',
 'SERAT',
 'GLNS',
 'UMPK',
 'ADSS',
 'AGPAT161',
 'OCBT',
 'G1PACT',
 'TMPK',
 'ACONTa',
 'ARGSS',
 'PSD160',
 'P5CR',
 'NDPK2',
 'HSK',
 'PE161abcpp',
 'ADSL1r',
 'ALAR',
 'COBALT2tpp',
 'ACGK',
 'ARGSL',
 'NH4tex',
 'GMPS2',
 'DASYN161',
 'NADS1',
 'THRS',
 'THRD_L',
 'PSD161',
 'PMPK',
 'ACPPAT160']

In [2965]:
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_above_zero):
    
    i=0
    r_id = inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1
    
w7


['APG3PAT160',
 'ACGS',
 'DTMPK',
 'ACODA',
 'APG3PAT161',
 '3OAS140',
 'HCO3E',
 'NADK',
 'NH4tpp',
 'CTPS2',
 'CS',
 'ACCOAC',
 'ACPPAT161',
 'O2tpp',
 'G5SADs',
 'AGPAT160',
 'GF6PTA',
 'GK1',
 'PSSA161',
 'ALAALAr',
 'ICDHyr',
 'NDPK4',
 'ASPK',
 'UAGDP',
 'PSSA160',
 'PE160abcpp',
 '3OAR140',
 'DASYN160',
 'CYSS',
 'ACONTb',
 'CU2tpp',
 'MCOATA',
 'NNATr',
 'SERAT',
 'GLNS',
 'UMPK',
 'ADSS',
 'AGPAT161',
 'OCBT',
 'G1PACT',
 'TMPK',
 'ACONTa',
 'ARGSS',
 'PSD160',
 'P5CR',
 'NDPK2',
 'HSK',
 'PE161abcpp',
 'ADSL1r',
 'ALAR',
 'COBALT2tpp',
 'ACGK',
 'ARGSL',
 'NH4tex',
 'GMPS2',
 'DASYN161',
 'NADS1',
 'THRS',
 'THRD_L',
 'PSD161',
 'PMPK',
 'ACPPAT160']

In [2966]:
w8

['3OAS140',
 'PSSA161',
 'UAGDP',
 'PSSA160',
 '3OAR140',
 'OCBT',
 'G1PACT',
 'ARGSS',
 'PSD160',
 'ARGSL',
 'THRD_L',
 'PSD161']

In [2967]:
set1= set(inANDdecr_above_zero)
set2= set(w7)
set3= set1-set2
inANDdecr_above_zero= list(set3)

set1= set(inANDdecr_above_zero)
set2= set(w8)
set3= set1-set2
inANDdecr_above_zero= list(set3)

decr_above_zero= list(decr_above_zero) + list(w7) + list(w8)


In [2968]:
decr_above_zero = list(set(decr_above_zero))


In [2969]:
i2= 0
r_id = diff_indecr_below_zero[0]
incr_below_zero= list()
decr_below_zero= list()
one_to_three_data_point_below_zero= list()

for r_id in diff_indecr_below_zero:                    #this code separates reactions with an increasing trend from reactions with a decreasing trend above zero
    
    i=0
    r_id = diff_indecr_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>3:
        
        if my_series.is_monotonic_decreasing ==True:
            decr_below_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_below_zero.append(r_id)
    
    elif len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4:
        
        one_to_three_data_point_below_zero.append(r_id)

    
    i2= i2 + 1

decr_below_zero


['EX_cmp_e',
 'EX_glcur1p_e',
 'EX_gmp_e',
 'EX_gsn_e',
 'EX_23camp_e',
 'EX_23cgmp_e',
 'EX_23cump_e',
 'EX_2ddglcn_e',
 'EX_3amp_e',
 'EX_3cmp_e',
 'EX_3gmp_e',
 'EX_3ump_e',
 'EX_cytd_e',
 'EX_dad_2_e',
 'EX_imp_e',
 'EX_acac_e',
 'EX_damp_e',
 'EX_ins_e',
 'EX_lcts_e',
 'EX_adn_e',
 'EX_dcmp_e',
 'EX_dcyt_e',
 'EX_mal__D_e',
 'EX_dgmp_e',
 'EX_man6p_e',
 'EX_amp_e',
 'EX_dgsn_e',
 'EX_dimp_e',
 'EX_din_e',
 'EX_dtmp_e',
 'EX_dump_e',
 'EX_duri_e',
 'EX_pser__L_e',
 'EX_rmn_e',
 'EX_ser__D_e',
 'EX_f6p_e',
 'EX_thrp_e',
 'EX_thymd_e',
 'EX_tre_e',
 'EX_trp__L_e',
 'EX_ump_e',
 'EX_uri_e',
 'EX_g3pc_e',
 'EX_g3pe_e',
 'EX_g3pg_e',
 'EX_g3pi_e',
 'EX_g3ps_e',
 'EX_g6p_e']

In [2970]:
one_to_three_data_point_below_zero

['EX_indole_e',
 'EX_pi_e',
 '12PPDRtex',
 '12PPDRtpp',
 '12PPDStex',
 '12PPDStpp',
 'EX_quin_e',
 'EX_skm_e',
 'EX_so4_e',
 'EX_taur_e',
 '3HPPtex',
 'ABUTtex',
 'ACACCT',
 'ACALDtex',
 'ACALDtpp',
 '5DGLCNR',
 '5DGLCNt2rpp',
 '5DGLCNtex',
 'AIRC3',
 'AKGt2rpp',
 'AKGtex',
 'ACSERtex',
 'ACtex',
 'ADEt2rpp',
 'ADEtex',
 'ALAtex',
 'ALCD19',
 'ALCD2x',
 'ALDD3y',
 'ALLTNt2rpp',
 'ALLTNtex',
 'ASPtex',
 'BALAt2pp',
 'BUTCT',
 'CITtex',
 'CSNt2pp',
 'DALAtex',
 'DDGLK',
 'CYStex',
 'DHAtex',
 'DHAtpp',
 'D_LACt2pp',
 'D_LACtex',
 'DHORTS',
 'DSERt2pp',
 'ECOAH4',
 'ETHAtex',
 'ETOHtex',
 'ETOHtrpp',
 'ASNtex',
 'GLCNtex',
 'GLYCAtex',
 'GLUR',
 'GLUtex',
 'GLYALDtex',
 'GLYALDtpp',
 'GUAtex',
 'GLYC3Ptex',
 'GLYCAt2rpp',
 'H2Stex',
 'GLYCLTtex',
 'HYXNtex',
 'HYXNtpp',
 'IDONt2rpp',
 'IDONtex',
 'IMPC',
 'GLYCtex',
 'HOMtex',
 'L_LACt2rpp',
 'GLYtex',
 'MALtex',
 'OROTt2_2pp',
 'ORPT',
 'PROtex',
 'PYRt2rpp',
 'PYRtex',
 'PSP_Lpp',
 'RBK_L1',
 'PTHRpp',
 'RIBabcpp',
 'SUCCtex',
 'SERtex'

In [2971]:
incr_below_zero

['EX_cpgn_e',
 'EX_isetac_e',
 'EX_arbtn_fe3_e',
 'EX_mso3_e',
 'EX_no_e',
 'EX_no2_e',
 'EX_dmso_e',
 'EX_no3_e',
 'EX_orot_e',
 'EX_sulfac_e',
 'EX_tmao_e',
 'EX_fe3dhbzs_e',
 '2DGULRGx',
 'EX_fe3hox_e',
 'EX_fecrm_e',
 'EX_feenter_e',
 'EX_feoxam_e',
 'ACOATA',
 'ADK4',
 'ALPATE160pp',
 'ALPATG160pp',
 'ATHRDHr',
 'CHLt2pp',
 'BETALDHy',
 'BSORx',
 'CTBTt2rpp',
 'CUt3',
 'ARGAGMt7pp',
 'DOGULNR',
 'EAR60x',
 'EAR80x',
 'DSERDHr',
 'ECOAH5',
 'ECOAH6',
 'ECOAH7',
 'ECOAH8',
 'EAR40x',
 'FESD2s',
 'FORtex',
 'ASNt2rpp',
 'FRUpts2pp',
 'FUMt2_2pp',
 'G2PPpp',
 'ASPO6',
 'GLNabcpp',
 'GALt2pp',
 'GLUNpp',
 'GLYBt2pp',
 'GUAtpp',
 'HEXt2rpp',
 'HG2t3pp',
 'ILEt2rpp',
 'HISt2rpp',
 'LACZ',
 'LPLIPAL2ATE120',
 'LPLIPAL2ATE140',
 'LPLIPAL2ATE141',
 'LPLIPAL2ATE160',
 'LPLIPAL2ATE161',
 'LPLIPAL2ATE180',
 'LPLIPAL2ATE181',
 'LPLIPAL2ATG120',
 'LPLIPAL2ATG140',
 'LPLIPAL2ATG141',
 'LPLIPAL2ATG160',
 'LPLIPAL2ATG161',
 'LPLIPAL2ATG180',
 'LPLIPAL2ATG181',
 'LEUt2rpp',
 'LIPOCT',
 'GPDDA3pp',
 

In [2972]:
set1= set(diff_indecr_below_zero)
set2= set(decr_below_zero)
set3= set(incr_below_zero)
set4= set(one_to_three_data_point_below_zero)
inANDdecr_below_zero= list(set1-set2-set3-set4)
inANDdecr_below_zero


['EX_23dappa_e',
 'EX_anhgm_e',
 'EX_LalaDgluMdapDala_e',
 'EX_nh4_e',
 'AGPR',
 'EX_glcr_e',
 'EX_melib_e',
 'EX_fe3dcit_e',
 'EX_ttdca_e',
 'EX_butso3_e',
 'EX_xylu__L_e',
 'EX_26dap__M_e',
 'EX_fum_e',
 'EX_23ccmp_e',
 'G3PD2',
 'EX_tartr__L_e',
 'EX_3hpppn_e',
 'EX_gthrd_e',
 'EX_ptrc_e',
 'EX_acmana_e',
 'ASAD',
 'EX_ocdca_e',
 'EX_tartr__D_e',
 'EX_ascb__L_e',
 'EX_ethso3_e',
 'EX_arab__L_e',
 'EX_hxa_e',
 'EX_udpglcur_e',
 'EX_LalaDgluMdap_e',
 'HSDy',
 'EX_galt_e',
 'EX_but_e',
 'EX_galur_e',
 'EX_galct__D_e',
 'EX_malt_e',
 'EX_octa_e',
 'EX_lyx__L_e',
 'EX_dca_e',
 'RPI',
 'EX_rib__D_e',
 'EX_all__D_e',
 'EX_hdcea_e',
 'EX_acnam_e',
 'ASPTA',
 'OPHHX3',
 'EX_xyl__D_e',
 'EX_progly_e',
 'OMPHHX3',
 'EX_pppn_e',
 'EX_LalaLglu_e',
 'EX_sbt__D_e',
 'EX_ttdcea_e',
 'EX_ddca_e',
 'EX_cgly_e',
 'OMMBLHX3',
 'EX_chtbs_e',
 'EX_udpacgal_e',
 'EX_3hcinnm_e',
 'EX_ocdcea_e',
 'EX_gal_e',
 'EX_acmum_e',
 'EX_LalaDglu_e',
 'EX_hdca_e',
 'EX_acgam_e',
 'PGAMT',
 'EX_mnl_e',
 'EX_sucr_e',
 

In [2973]:
w9= list()
w10= list()
i2=0

while i2 < len(inANDdecr_below_zero):
    
    i=0
    r_id = inANDdecr_below_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w9.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w10.append(r_id)
    
    i2= i2+1
    
w9


['PGAMT']

In [2974]:
w10

['EX_nh4_e',
 'AGPR',
 'EX_26dap__M_e',
 'G3PD2',
 'ASAD',
 'HSDy',
 'RPI',
 'ASPTA',
 'OPHHX3',
 'OMPHHX3',
 'OMMBLHX3',
 'PGAMT',
 'ACOTA']

In [2975]:
set1= set(inANDdecr_below_zero)
set2= set(w10)
inANDdecr_below_zero= list(set1-set2)

incr_below_zero= list(incr_below_zero) + list(w10)


In [2976]:
i=0
inANDdecr_decr_below_zero_and_incr_above_zero= list()
inANDdecr_decr_below_zero_and_decr_above_zero= list()
inANDdecr_incr_below_zero_and_decr_above_zero= list()
inANDdecr_incr_below_zero_and_incr_above_zero= list()
inANDdecr_decr_below_zero_and_inANDdecr_above_zero= list()
inANDdecr_incr_below_zero_and_inANDdecr_above_zero= list()
inANDdecr_inANDdecr_below_zero_and_decr_above_zero= list()
inANDdecr_inANDdecr_below_zero_and_incr_above_zero= list()

while i < len(inANDdecr_both_positive_and_negative):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(inANDdecr_both_positive_and_negative[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic_decreasing ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_decr_below_zero_and_decr_above_zero:
            inANDdecr_decr_below_zero_and_decr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_incr_below_zero_and_incr_above_zero:
            inANDdecr_incr_below_zero_and_incr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic_decreasing ==True and my_series2.is_monotonic ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_decr_below_zero_and_incr_above_zero:
            inANDdecr_decr_below_zero_and_incr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic ==True and my_series2.is_monotonic_decreasing ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_incr_below_zero_and_decr_above_zero:
            inANDdecr_incr_below_zero_and_decr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic_decreasing ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_decr_below_zero_and_inANDdecr_above_zero:
            inANDdecr_decr_below_zero_and_inANDdecr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series.is_monotonic ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_incr_below_zero_and_inANDdecr_above_zero:
            inANDdecr_incr_below_zero_and_inANDdecr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series2.is_monotonic_decreasing ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_inANDdecr_below_zero_and_decr_above_zero:
            inANDdecr_inANDdecr_below_zero_and_decr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        elif my_series2.is_monotonic ==True and inANDdecr_both_positive_and_negative[i] not in inANDdecr_inANDdecr_below_zero_and_incr_above_zero:
            inANDdecr_inANDdecr_below_zero_and_incr_above_zero.append(inANDdecr_both_positive_and_negative[i])
        
    i=i+1

inANDdecr_decr_below_zero_and_decr_above_zero


[]

In [2977]:
inANDdecr_incr_below_zero_and_inANDdecr_above_zero

['GAPD', 'SUCDi', 'ENO', 'EX_co2_e']

In [2978]:
inANDdecr_inANDdecr_below_zero_and_decr_above_zero

['PGM', 'THD2pp', 'CO2tpp', 'GLUDy', 'PPS', 'PGK', 'CO2tex', 'FBP', 'PPCK']

In [2979]:
inANDdecr_incr_below_zero_and_decr_above_zero

['FACOAE120',
 'GDPTPDP',
 'EX_val__L_e',
 '2AGPGAT120',
 '2AGPGAT180',
 'CPGNR2',
 'ACALD',
 'FACOAE160',
 'MGSA',
 'TKT2',
 'NMNN',
 'ASNS2',
 'CLPNH180pp',
 'FE3HOXR1',
 '2AGPEAT180',
 'PPC',
 'PTRCt2pp',
 'DURIPP',
 'AACPS4',
 'CDAPPA141',
 'FEOXAMR2',
 'PGPP180',
 'CPGNR3',
 'NI2t3pp',
 'AACPS3',
 'PGPP140',
 'RPE',
 'PUNP2',
 'SUCASPtpp',
 'PGPP160',
 'LPLIPAL2E140',
 'FACOAE181',
 'G3PGabcpp',
 'MTHFD',
 'ORNDC',
 'GTHRDabcpp',
 'FACOAE140',
 'PA120abcpp',
 'RNDR3b',
 'FE2tpp',
 '2AGPEAT160',
 'CLPNH120pp',
 'CYSDS',
 'GLYK',
 'THRt4pp',
 'FA100ACPHi',
 '2AGPGAT181',
 'RNDR3',
 'ZN2tpp',
 'CDAPPA160',
 'PPK2',
 '2AGPEAT161',
 'GHMT2r',
 'RNDR1b',
 'FEOXAMR3',
 'NACODA',
 'LPLIPAL2G120',
 'TMDK1',
 'FECRMR1',
 'FACOAE141',
 'PA160abcpp',
 '2AGPGAT140',
 'RNTR1c',
 'P5CD',
 '2AGPGAT160',
 'GDPDPK',
 'UM3PL',
 'LPLIPAL2G161',
 'PROD2',
 'CLPNH141pp',
 '2AGPGAT161',
 'MN2tpp',
 'LYSt2pp',
 'TPI',
 'HXAND',
 'ICL',
 'RNDR2b',
 'GLYCL',
 'CDAPPA140',
 '2AGPGAT141',
 'PGPP141',
 'ARBTN

In [2980]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_incr_below_zero_and_inANDdecr_above_zero):
    
    i=0
    r_id = inANDdecr_incr_below_zero_and_inANDdecr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 101:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 100
    l= list()
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


['SUCDi']

In [2981]:
w7

['GAPD', 'SUCDi', 'ENO', 'EX_co2_e']

In [2982]:
w8

['GAPD']

In [2983]:
set1= set(inANDdecr_incr_below_zero_and_inANDdecr_above_zero)
set2= set(w7)
inANDdecr_incr_below_zero_and_inANDdecr_above_zero= list(set1-set2)

inANDdecr_incr_below_zero_and_decr_above_zero= list(inANDdecr_incr_below_zero_and_decr_above_zero)+list(w7)


In [2984]:
w5= list()
w6= list()
w7= list()
w8= list()
i2=0

while i2 < len(inANDdecr_inANDdecr_below_zero_and_decr_above_zero):
    
    i=0
    r_id = inANDdecr_inANDdecr_below_zero_and_decr_above_zero[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(r_id))
    l= list()

    while i < 101:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w5.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w6.append(r_id)
    
    
    i= 100
    l= list()
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    l= list(csvFile2.Biomass)

    n2=0

    if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and l[n2+3]<max(l) and (l[n2+4] < max(l) or l[n2+4] == max(l)):
        n2=0
    elif l[n2] == max(l):
        n2=0
    else:
        w7.append(r_id)

    
    n2= len(l)-1

    if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and l[n2-3]<max(l) and (l[n2-4] < max(l) or l[n2-4] == max(l)):
        n2= len(l)-1
    elif l[n2] == max(l):
        n2= len(l)-1
    else:
        w8.append(r_id)
    
    i2= i2+1

w5


['PPS', 'PGK']

In [2985]:
w6

['PGM', 'THD2pp', 'CO2tpp', 'GLUDy', 'PPS', 'PGK', 'CO2tex', 'FBP', 'PPCK']

In [2986]:
set1= set(inANDdecr_inANDdecr_below_zero_and_decr_above_zero)
set2= set(w6)
inANDdecr_inANDdecr_below_zero_and_decr_above_zero= list(set1-set2)

inANDdecr_incr_below_zero_and_decr_above_zero= list(inANDdecr_incr_below_zero_and_decr_above_zero)+list(w6)


In [2987]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()

while i < len(flat_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(flat_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(flat_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(flat_above_zero[i])
    
    i= i + 1

one_to_three_data_point_below_zero12

set1= set(flat_above_zero)
set2= set(one_to_three_data_point_above_zero12)
flat_above_zero= list(set1-set2)
one_to_three_data_point_above_zero.append("ACLS")
one_to_three_data_point_above_zero.append("DHAD1")
one_to_three_data_point_above_zero


['DM_oxam_c',
 'EX_12ppd__R_e',
 'EX_h2s_e',
 'EX_3hpp_e',
 'EX_hom__L_e',
 'EX_acser_e',
 'EX_cit_e',
 'EX_thym_e',
 '3HAD100',
 '3AMACHYD',
 '3HAD121',
 '3HAD141',
 '3HPPtpp',
 '3OAR100',
 '3OAR121',
 '3OAR141',
 '3OAS100',
 '3OAS121',
 '3OAS141',
 'ACGAMK',
 '3PEPTabcpp',
 'ACM6PH',
 'ACOAD4f',
 'AICART',
 'AIRC2',
 'ACSERtpp',
 'ALAGLUE',
 'ALLTAMH',
 'ALLTN',
 'ADSK',
 'ADSL2r',
 'AGM4PApp',
 'ALR2',
 'ALR4x',
 'AM3PA',
 'AGM4PH',
 'AGM4Pt2pp',
 'AGMH',
 'AGMt2pp',
 'ANHMK',
 'BPNT',
 'CITt3pp',
 'CBMD',
 'DHFR',
 'G1SAT',
 'ASPCT',
 'GLCDpp',
 'GLCNt2rpp',
 'GLUPRT',
 'GLUTRR',
 'GLUTRS',
 'H2St1pp',
 'HOMt2pp',
 'LPLIPAL1E181pp',
 'LADGMDH',
 'LALGP',
 'LCARR',
 'LCARS',
 'LPLIPAL1E120pp',
 'LPLIPAL1E140pp',
 'LPLIPAL1E141pp',
 'LPLIPAL1E160pp',
 'LPLIPAL1E161pp',
 'LPLIPAL1E180pp',
 'MCTP1App',
 'MCTP1Bpp',
 'MDDCP1pp',
 'MDDCP4pp',
 'MDDCP5pp',
 'MDDEP1pp',
 'MDDEP2pp',
 'MLDCP1App',
 'MLDCP3App',
 'MLDEP1pp',
 'GPDDA2pp',
 'MLTGY3pp',
 'MPTG',
 'MSAR',
 'OMPDC',
 'OXAMTC',
 '

In [2988]:
one_to_three_data_point_above_zero12

['ACLS', 'DHAD1']

In [2989]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()

while i < len(flat_below_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(flat_below_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(flat_below_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(flat_below_zero[i])
    
    i= i + 1

one_to_three_data_point_below_zero12

set1= set(flat_below_zero)
set2= set(one_to_three_data_point_below_zero12)
flat_below_zero= list(set1-set2)
one_to_three_data_point_below_zero.append("KARA1")
one_to_three_data_point_below_zero


['EX_indole_e',
 'EX_pi_e',
 '12PPDRtex',
 '12PPDRtpp',
 '12PPDStex',
 '12PPDStpp',
 'EX_quin_e',
 'EX_skm_e',
 'EX_so4_e',
 'EX_taur_e',
 '3HPPtex',
 'ABUTtex',
 'ACACCT',
 'ACALDtex',
 'ACALDtpp',
 '5DGLCNR',
 '5DGLCNt2rpp',
 '5DGLCNtex',
 'AIRC3',
 'AKGt2rpp',
 'AKGtex',
 'ACSERtex',
 'ACtex',
 'ADEt2rpp',
 'ADEtex',
 'ALAtex',
 'ALCD19',
 'ALCD2x',
 'ALDD3y',
 'ALLTNt2rpp',
 'ALLTNtex',
 'ASPtex',
 'BALAt2pp',
 'BUTCT',
 'CITtex',
 'CSNt2pp',
 'DALAtex',
 'DDGLK',
 'CYStex',
 'DHAtex',
 'DHAtpp',
 'D_LACt2pp',
 'D_LACtex',
 'DHORTS',
 'DSERt2pp',
 'ECOAH4',
 'ETHAtex',
 'ETOHtex',
 'ETOHtrpp',
 'ASNtex',
 'GLCNtex',
 'GLYCAtex',
 'GLUR',
 'GLUtex',
 'GLYALDtex',
 'GLYALDtpp',
 'GUAtex',
 'GLYC3Ptex',
 'GLYCAt2rpp',
 'H2Stex',
 'GLYCLTtex',
 'HYXNtex',
 'HYXNtpp',
 'IDONt2rpp',
 'IDONtex',
 'IMPC',
 'GLYCtex',
 'HOMtex',
 'L_LACt2rpp',
 'GLYtex',
 'MALtex',
 'OROTt2_2pp',
 'ORPT',
 'PROtex',
 'PYRt2rpp',
 'PYRtex',
 'PSP_Lpp',
 'RBK_L1',
 'PTHRpp',
 'RIBabcpp',
 'SUCCtex',
 'SERtex'

In [2990]:
one_to_three_data_point_above_zero12

[]

In [2991]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()

while i < len(incr_and_stable_below_zero_and_inANDdecr_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(incr_and_stable_below_zero_and_inANDdecr_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(incr_and_stable_below_zero_and_inANDdecr_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(incr_and_stable_below_zero_and_inANDdecr_above_zero[i])
    
    i= i + 1

one_to_three_data_point_below_zero12

set1= set(incr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(one_to_three_data_point_above_zero12)
incr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2)
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero12)
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero


[]

In [2992]:
one_to_three_data_point_above_zero12

[]

In [2993]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()

while i < len(inANDdecr_incr_below_zero_and_decr_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(inANDdecr_incr_below_zero_and_decr_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(inANDdecr_incr_below_zero_and_decr_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(inANDdecr_incr_below_zero_and_decr_above_zero[i])
    
    i= i + 1

one_to_three_data_point_below_zero12

set1= set(inANDdecr_incr_below_zero_and_decr_above_zero)
set2= set(one_to_three_data_point_above_zero12)
inANDdecr_incr_below_zero_and_decr_above_zero= list(set1-set2)
inANDdecr_incr_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero12)
inANDdecr_incr_below_zero_and_one_to_three_data_point_above_zero


['EX_val__L_e',
 'G3PGabcpp',
 'UM3PL',
 'TPI',
 'PAPSR',
 'PAPSR2',
 'G3PEabcpp',
 'GLYCtpp',
 'ENTERES2',
 'GARFT',
 'EX_ura_e',
 'UGLYCH',
 'AGM4PA',
 'GAPD']

In [2994]:
set1= set(inANDdecr_incr_below_zero_and_decr_above_zero)
set2= set(one_to_three_data_point_below_zero12)
inANDdecr_incr_below_zero_and_decr_above_zero= list(set1-set2)
one_to_three_data_point_below_zero_and_mon_decr_above_zero= list(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+list(one_to_three_data_point_below_zero12)
one_to_three_data_point_below_zero_and_mon_decr_above_zero


['FUM',
 'TKT2',
 'DURIPP',
 'CYSDS',
 'TMDK1',
 'FACOAE141',
 'P5CD',
 'PROD2',
 'ICL',
 'FA80ACPHi',
 'DALAt2pp',
 'TKT1',
 'CYSabcpp',
 'NTD1',
 'XPPT',
 'AACPS2',
 'GMPR',
 'PGL',
 'SUCDi',
 'PGK']

In [2995]:
one_to_three_data_point_above_zero12

['EX_val__L_e',
 'G3PGabcpp',
 'UM3PL',
 'TPI',
 'PAPSR',
 'PAPSR2',
 'G3PEabcpp',
 'GLYCtpp',
 'ENTERES2',
 'GARFT',
 'EX_ura_e',
 'UGLYCH',
 'AGM4PA',
 'GAPD']

In [2996]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()

while i < len(inANDdecr_and_stable_below_zero_and_decreasing_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(inANDdecr_and_stable_below_zero_and_decreasing_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(inANDdecr_and_stable_below_zero_and_decreasing_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(inANDdecr_and_stable_below_zero_and_decreasing_above_zero[i])
    
    i= i + 1

one_to_three_data_point_below_zero12

set1= set(inANDdecr_and_stable_below_zero_and_decreasing_above_zero)
set2= set(one_to_three_data_point_above_zero12)
inANDdecr_and_stable_below_zero_and_decreasing_above_zero= list(set1-set2)
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero12)
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero


['EX_ac_e']

In [2997]:
one_to_three_data_point_above_zero12

['EX_ac_e']

In [2998]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()
decreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()

while i < len(decreasing_and_stable_below_zero_and_decreasing_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(decreasing_and_stable_below_zero_and_decreasing_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(decreasing_and_stable_below_zero_and_decreasing_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(decreasing_and_stable_below_zero_and_decreasing_above_zero[i])
    
    i= i + 1

one_to_three_data_point_below_zero12

set1= set(decreasing_and_stable_below_zero_and_decreasing_above_zero)
set2= set(one_to_three_data_point_above_zero12)
decreasing_and_stable_below_zero_and_decreasing_above_zero= list(set1-set2)
decreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero12)
decreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero


['EX_glcn_e',
 'EX_glyclt_e',
 'EX_hxan_e',
 'EX_cys__L_e',
 'GLCabcpp',
 'SULabcpp',
 'H2SO']

In [2999]:
one_to_three_data_point_above_zero12

['EX_glcn_e',
 'EX_glyclt_e',
 'EX_hxan_e',
 'EX_cys__L_e',
 'GLCabcpp',
 'SULabcpp',
 'H2SO']

In [3000]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()
inANDdecr_below_zero_and_one_to_three_data_point_above_zero= list()

while i < len(inANDdecr_below_zero_and_decr_and_stable_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(inANDdecr_below_zero_and_decr_and_stable_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(inANDdecr_below_zero_and_decr_and_stable_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(inANDdecr_below_zero_and_decr_and_stable_above_zero[i])
    
    i= i + 1

one_to_three_data_point_below_zero12

set1= set(inANDdecr_below_zero_and_decr_and_stable_above_zero)
set2= set(one_to_three_data_point_above_zero12)
inANDdecr_below_zero_and_decr_and_stable_above_zero= list(set1-set2)
inANDdecr_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero12)
inANDdecr_below_zero_and_one_to_three_data_point_above_zero


['EX_pro__L_e',
 'EX_idon__L_e',
 'EX_gua_e',
 'EX_xan_e',
 'EX_glyc_e',
 'EX_12ppd__S_e',
 'EX_4abut_e',
 'EX_ade_e',
 'EX_asn__L_e',
 'EX_glyc__R_e',
 'EX_alltn_e',
 'EX_etoh_e']

In [3001]:
one_to_three_data_point_above_zero12

['EX_pro__L_e',
 'EX_idon__L_e',
 'EX_gua_e',
 'EX_xan_e',
 'EX_glyc_e',
 'EX_12ppd__S_e',
 'EX_4abut_e',
 'EX_ade_e',
 'EX_asn__L_e',
 'EX_glyc__R_e',
 'EX_alltn_e',
 'EX_etoh_e']

In [3002]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()
incr_below_zero_and_one_to_three_data_point_above_zero= list()

while i < len(incr_below_zero_and_incr_and_stable_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(incr_below_zero_and_incr_and_stable_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(incr_below_zero_and_incr_and_stable_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(incr_below_zero_and_incr_and_stable_above_zero[i])
    
    i= i + 1

one_to_three_data_point_below_zero12

set1= set(incr_below_zero_and_incr_and_stable_above_zero)
set2= set(one_to_three_data_point_above_zero12)
incr_below_zero_and_incr_and_stable_above_zero= list(set1-set2)
incr_below_zero_and_one_to_three_data_point_above_zero= list(incr_below_zero_and_one_to_three_data_point_above_zero)+list(one_to_three_data_point_above_zero12)
incr_below_zero_and_one_to_three_data_point_above_zero


['GLCptspp']

In [3003]:
one_to_three_data_point_above_zero12

['GLCptspp']

In [3004]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()
decr_below_zero_and_one_to_three_data_point_above_zero= list()

while i < len(decr_below_zero_and_decr_and_stable_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(decr_below_zero_and_decr_and_stable_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(decr_below_zero_and_decr_and_stable_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(decr_below_zero_and_decr_and_stable_above_zero[i])
    
    i= i + 1

one_to_three_data_point_below_zero12

set1= set(decr_below_zero_and_decr_and_stable_above_zero)
set2= set(one_to_three_data_point_above_zero12)
decr_below_zero_and_decr_and_stable_above_zero= list(set1-set2)
decr_below_zero_and_one_to_three_data_point_above_zero= list(decr_below_zero_and_one_to_three_data_point_above_zero)+list(one_to_three_data_point_above_zero12)
decr_below_zero_and_one_to_three_data_point_above_zero


['EX_glu__L_e',
 'EX_gly_e',
 'EX_glyald_e',
 'EX_glyc3p_e',
 'EX_5dglcn_e',
 'EX_acald_e',
 'EX_lac__D_e',
 'EX_lac__L_e',
 'EX_akg_e',
 'EX_ala__L_e',
 'EX_mal__L_e',
 'EX_asp__L_e',
 'EX_dha_e',
 'EX_pyr_e',
 'EX_ser__L_e',
 'EX_etha_e',
 'EX_thr__L_e',
 'EX_ala__D_e',
 'EX_succ_e']

In [3005]:
one_to_three_data_point_above_zero12

['EX_glu__L_e',
 'EX_gly_e',
 'EX_glyald_e',
 'EX_glyc3p_e',
 'EX_5dglcn_e',
 'EX_acald_e',
 'EX_lac__D_e',
 'EX_lac__L_e',
 'EX_akg_e',
 'EX_ala__L_e',
 'EX_mal__L_e',
 'EX_asp__L_e',
 'EX_dha_e',
 'EX_pyr_e',
 'EX_ser__L_e',
 'EX_etha_e',
 'EX_thr__L_e',
 'EX_ala__D_e',
 'EX_succ_e']

In [3006]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()

while i < len(incr_below_zero_and_decr_and_stable_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(incr_below_zero_and_decr_and_stable_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(incr_below_zero_and_decr_and_stable_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(incr_below_zero_and_decr_and_stable_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero12

set1= set(incr_below_zero_and_decr_and_stable_above_zero)
set2= set(one_to_three_data_point_above_zero12)
incr_below_zero_and_decr_and_stable_above_zero= list(set1-set2)
incr_below_zero_and_one_to_three_data_point_above_zero= list(incr_below_zero_and_one_to_three_data_point_above_zero)+list(one_to_three_data_point_above_zero12)
incr_below_zero_and_one_to_three_data_point_above_zero


['GLCptspp',
 'EAR100x',
 'EAR121x',
 'EAR141x',
 'GLCt2pp',
 'IDOND',
 'LALDO2x',
 'SO4t2pp']

In [3007]:
one_to_three_data_point_above_zero12

['EAR100x', 'EAR121x', 'EAR141x', 'GLCt2pp', 'IDOND', 'LALDO2x', 'SO4t2pp']

In [3008]:
incr_and_stable_below_zero_and_flat_above_zero

['ACACT4r',
 'ACACT5r',
 'ACACT6r',
 'ACACT7r',
 'FE2tex',
 'HACD4',
 'HACD5',
 'HACD6',
 'HACD7',
 'HACD8',
 'Htex',
 'EX_fe3_e']

In [3009]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()
one_to_three_data_point_below_zero_and_flat_above_zero= list()

while i < len(incr_and_stable_below_zero_and_flat_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(incr_and_stable_below_zero_and_flat_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(incr_and_stable_below_zero_and_flat_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(incr_and_stable_below_zero_and_flat_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero12

set1= set(incr_and_stable_below_zero_and_flat_above_zero)
set2= set(one_to_three_data_point_below_zero12)
incr_and_stable_below_zero_and_flat_above_zero= list(set1-set2)
one_to_three_data_point_below_zero_and_flat_above_zero= list(one_to_three_data_point_below_zero12)
one_to_three_data_point_below_zero_and_flat_above_zero


['ACACT4r', 'HACD4']

In [3010]:
incr_and_stable_below_zero_and_flat_above_zero

['ACACT5r',
 'ACACT7r',
 'HACD5',
 'HACD8',
 'HACD7',
 'ACACT6r',
 'Htex',
 'HACD6',
 'EX_fe3_e',
 'FE2tex']

In [3011]:
one_to_three_data_point_above_zero12

[]

In [3012]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()


while i < len(flat_below_zero_and_decr_and_stable_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(flat_below_zero_and_decr_and_stable_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(flat_below_zero_and_decr_and_stable_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(flat_below_zero_and_decr_and_stable_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero12

set1= set(flat_below_zero_and_decr_and_stable_above_zero)
set2= set(one_to_three_data_point_above_zero12)
flat_below_zero_and_decr_and_stable_above_zero= list(set1-set2)
flat_below_zero_and_one_to_three_data_point_above_zero= list(one_to_three_data_point_above_zero12)
flat_below_zero_and_one_to_three_data_point_above_zero


['CTECOAI6_copy1',
 'DHORD2',
 'DHORDfum',
 'EAR100y',
 'EAR121y',
 'EAR141y',
 'MDDCP3pp',
 'MLDCP2App']

In [3013]:
i=0
one_to_three_data_point_below_zero12= list()
one_to_three_data_point_above_zero12= list()
flat_below_zero_and_one_to_three_data_point_above_zero2= list()


while i < len(flat_below_zero_and_flat_above_zero):
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}.csv'.format(flat_below_zero_and_flat_above_zero[i]))
    n= 0
    l= list()
    
    while n < 101:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n2= 100
    l2= list()
    
    while n2 < 201:
        inf= csvFile.iloc[n2,  2]
        flt= float(csvFile.iloc[n2,  2])
    
        if csvFile.iloc[n2,  2] == flt:
            l2.append(csvFile.iloc[n2,  2])
        n2= n2+1

    csvFile3= pd.DataFrame (l2, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()
    
    if len(csvFile2['Biomass'])>1 and len(csvFile2['Biomass'])<4 and r_id not in one_to_three_data_point_below_zero12:
        
        one_to_three_data_point_below_zero12.append(flat_below_zero_and_flat_above_zero[i])

    if len(csvFile3['Biomass'])>1 and len(csvFile3['Biomass'])<4 and r_id not in one_to_three_data_point_above_zero12:
        
        one_to_three_data_point_above_zero12.append(flat_below_zero_and_flat_above_zero[i])
    
    i= i + 1
    
one_to_three_data_point_below_zero12

flat_below_zero_and_flat_above_zero.remove("GLCtex_copy1")
flat_below_zero_and_one_to_three_data_point_above_zero2= list(one_to_three_data_point_above_zero12)

flat_below_zero_and_one_to_three_data_point_above_zero= list(flat_below_zero_and_one_to_three_data_point_above_zero)+list(flat_below_zero_and_one_to_three_data_point_above_zero2)
flat_below_zero_and_one_to_three_data_point_above_zero


['CTECOAI6_copy1',
 'DHORD2',
 'DHORDfum',
 'EAR100y',
 'EAR121y',
 'EAR141y',
 'MDDCP3pp',
 'MLDCP2App',
 'GLCtex_copy1']

In [3014]:
one_to_three_data_point_above_zero12

['GLCtex_copy1']

In [3015]:
len(one_at_zero)+len(flat_above_zero)+len(flat_below_zero)+len(one_to_three_data_point_both_positive_and_negative)+len(flat_below_zero_and_flat_above_zero)+len(mon_decr_below_zero_and_flat_above_zero)+len(mon_incr_below_zero_and_flat_above_zero)+len(inANDdecr_below_zero_and_flat_above_zero)+len(flat_below_zero_and_mon_decr_above_zero)+len(flat_below_zero_and_mon_incr_above_zero)+len(flat_below_zero_and_inANDdecr_above_zero)+len(flat_below_zero_and_decr_and_stable_above_zero)+len(flat_below_zero_and_incr_and_stable_above_zero)+len(flat_below_zero_and_inANDdecr_and_stable_above_zero)+len(incr_and_stable_below_zero_and_flat_above_zero)+len(decr_and_stable_below_zero_and_flat_above_zero)+len(decr_both_positive_and_negative)+len(incr_both_positive_and_negative)+len(incr_below_zero_and_decr_and_stable_above_zero)+len(decr_below_zero_and_decr_and_stable_above_zero)+len(incr_below_zero_and_incr_and_stable_above_zero)+len(incr_below_zero_and_inANDdecr_and_stable_above_zero)+len(inANDdecr_below_zero_and_decr_and_stable_above_zero)+len(decreasing_and_stable_below_zero_and_decreasing_above_zero)+len(increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_decreasing_above_zero)+len(decr_and_stable_below_zero_and_mon_decr_above_zero)+len(decr_above_zero)+len(one_to_three_data_point_above_zero)+len(inANDdecr_above_zero)+len(decr_below_zero)+len(incr_below_zero)+len(one_to_three_data_point_below_zero)+len(inANDdecr_below_zero)+len(inANDdecr_incr_below_zero_and_decr_above_zero)+len(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)+len(incr_and_stable_below_zero_and_inANDdecr_above_zero)+len(decr_and_stable_below_zero_and_inANDdecr_above_zero)+len(mon_decr_and_stable_below_zero)+len(mon_incr_and_stable_below_zero)+len(mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero)+len(inANDdecr_and_stable_above_zero)+len(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(inANDdecr_incr_below_zero_and_one_to_three_data_point_above_zero)+len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(decreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)+len(decr_below_zero_and_one_to_three_data_point_above_zero)+len(incr_below_zero_and_one_to_three_data_point_above_zero)+len(one_to_three_data_point_below_zero_and_flat_above_zero)+len(flat_below_zero_and_one_to_three_data_point_above_zero)+len(incr_and_stable_below_zero_and_mon_decr_above_zero)


2726

In [3016]:
lt= list(one_at_zero)+list(flat_above_zero)+list(flat_below_zero)+list(one_to_three_data_point_both_positive_and_negative)+list(flat_below_zero_and_flat_above_zero)+list(mon_decr_below_zero_and_flat_above_zero)+list(mon_incr_below_zero_and_flat_above_zero)+list(inANDdecr_below_zero_and_flat_above_zero)+list(flat_below_zero_and_mon_decr_above_zero)+list(flat_below_zero_and_mon_incr_above_zero)+list(flat_below_zero_and_inANDdecr_above_zero)+list(flat_below_zero_and_decr_and_stable_above_zero)+list(flat_below_zero_and_incr_and_stable_above_zero)+list(flat_below_zero_and_inANDdecr_and_stable_above_zero)+list(incr_and_stable_below_zero_and_flat_above_zero)+list(decr_and_stable_below_zero_and_flat_above_zero)+list(decr_both_positive_and_negative)+list(incr_both_positive_and_negative)+list(incr_below_zero_and_decr_and_stable_above_zero)+list(decr_below_zero_and_decr_and_stable_above_zero)+list(incr_below_zero_and_incr_and_stable_above_zero)+list(incr_below_zero_and_inANDdecr_and_stable_above_zero)+list(inANDdecr_below_zero_and_decr_and_stable_above_zero)+list(decreasing_and_stable_below_zero_and_decreasing_above_zero)+list(increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_decreasing_above_zero)+list(decr_and_stable_below_zero_and_mon_decr_above_zero)+list(decr_above_zero)+list(one_to_three_data_point_above_zero)+list(inANDdecr_above_zero)+list(decr_below_zero)+list(incr_below_zero)+list(one_to_three_data_point_below_zero)+list(inANDdecr_below_zero)+list(inANDdecr_incr_below_zero_and_decr_above_zero)+list(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+list(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)+list(incr_and_stable_below_zero_and_inANDdecr_above_zero)+list(decr_and_stable_below_zero_and_inANDdecr_above_zero)+list(mon_decr_and_stable_below_zero)+list(mon_incr_and_stable_below_zero)+list(mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero)+list(inANDdecr_and_stable_above_zero)+list(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(inANDdecr_incr_below_zero_and_one_to_three_data_point_above_zero)+list(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(decreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)+list(incr_below_zero_and_one_to_three_data_point_above_zero)+list(decr_below_zero_and_one_to_three_data_point_above_zero)+list(one_to_three_data_point_below_zero_and_flat_above_zero)+list(flat_below_zero_and_one_to_three_data_point_above_zero)+list(incr_and_stable_below_zero_and_mon_decr_above_zero)


In [3017]:
l1=[]
for i in lt:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')


In [3018]:
csvFile = pd.read_csv('all reactions.csv')
csvFile["Reaction names"]

rl= list(csvFile["Reaction names"])

set1= set(rl)
set2= set(lt)
set3= set1-set2
set3


set()

In [ ]:
decreasing_and_stable_below_zero_and_decreasing_above_zero= list(decreasing_and_stable_below_zero_and_decreasing_above_zero)+list(decr_and_stable_below_zero_and_mon_decr_above_zero)


In [3019]:
D= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
D1= pd.DataFrame (flat_above_zero, columns = ['Reaction names'])
D2= pd.DataFrame (flat_below_zero, columns = ['Reaction names'])
D3= pd.DataFrame (one_to_three_data_point_both_positive_and_negative, columns = ['Reaction names'])
D4= pd.DataFrame (flat_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D5= pd.DataFrame (mon_decr_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D6= pd.DataFrame (mon_incr_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D7= pd.DataFrame (inANDdecr_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D8= pd.DataFrame (flat_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D9= pd.DataFrame (flat_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D10= pd.DataFrame (flat_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D11= pd.DataFrame (flat_below_zero_and_decr_and_stable_above_zero, columns = ['Reaction names'])
D12= pd.DataFrame (flat_below_zero_and_incr_and_stable_above_zero, columns = ['Reaction names'])
D13= pd.DataFrame (flat_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D14= pd.DataFrame (incr_and_stable_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D15= pd.DataFrame (decr_and_stable_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D16= pd.DataFrame (decr_both_positive_and_negative, columns = ['Reaction names'])
D17= pd.DataFrame (incr_both_positive_and_negative, columns = ['Reaction names'])
D18= pd.DataFrame (incr_below_zero_and_decr_and_stable_above_zero, columns = ['Reaction names'])
D19= pd.DataFrame (decr_below_zero_and_decr_and_stable_above_zero, columns = ['Reaction names'])
D20= pd.DataFrame (incr_below_zero_and_incr_and_stable_above_zero, columns = ['Reaction names'])
D21= pd.DataFrame (incr_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D22= pd.DataFrame (inANDdecr_below_zero_and_decr_and_stable_above_zero, columns = ['Reaction names'])
D23= pd.DataFrame (decreasing_and_stable_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D24= pd.DataFrame (increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero, columns = ['Reaction names'])
D25= pd.DataFrame (inANDdecr_and_stable_below_zero_and_decreasing_above_zero, columns = ['Reaction names'])
D27= pd.DataFrame (decr_above_zero, columns = ['Reaction names'])
D28= pd.DataFrame (one_to_three_data_point_above_zero, columns = ['Reaction names'])
D29= pd.DataFrame (incr_above_zero, columns = ['Reaction names'])
D30= pd.DataFrame (inANDdecr_above_zero, columns = ['Reaction names'])
D32= pd.DataFrame (incr_below_zero, columns = ['Reaction names'])
D33= pd.DataFrame (decr_below_zero, columns = ['Reaction names'])
D34= pd.DataFrame (one_to_three_data_point_below_zero, columns = ['Reaction names'])
D35= pd.DataFrame (inANDdecr_below_zero, columns = ['Reaction names'])
D37= pd.DataFrame (inANDdecr_incr_below_zero_and_decr_above_zero, columns = ['Reaction names'])
D38= pd.DataFrame (one_to_three_data_point_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D39= pd.DataFrame (inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D40= pd.DataFrame (incr_and_stable_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D41= pd.DataFrame (decr_and_stable_below_zero_and_inANDdecr_above_zero, columns= ['Reaction names'])
D42= pd.DataFrame (mon_decr_and_stable_below_zero, columns = ['Reaction names'])
D43= pd.DataFrame (mon_incr_and_stable_below_zero, columns = ['Reaction names'])
D44= pd.DataFrame (mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D45= pd.DataFrame (mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D46= pd.DataFrame (inANDdecr_and_stable_below_zero, columns = ['Reaction names'])
D47= pd.DataFrame (inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D48= pd.DataFrame (incr_and_stable_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D49= pd.DataFrame (inANDdecr_incr_below_zero_and_one_to_three_data_point_above_zero, columns= ['Reaction names'])
D50= pd.DataFrame (inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D51= pd.DataFrame (decreasing_and_stable_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D52= pd.DataFrame (inANDdecr_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D53= pd.DataFrame (decr_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D54= pd.DataFrame (incr_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D55= pd.DataFrame (one_to_three_data_point_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D56= pd.DataFrame (flat_below_zero_and_one_to_three_data_point_above_zero, columns= ['Reaction names'])


In [ ]:
D.to_csv("NEW one feasible value at zero reactions.csv")
D1.to_csv("NEW flat_above_zero.csv")
D2.to_csv("NEW flat_below_zero.csv")
D3.to_csv("NEW one_to_three_data_point_both_positive_and_negative.csv")
D4.to_csv("NEW flat below and flat above zero.csv")
D5.to_csv("NEW increasing below zero and flat above zero.csv")
D6.to_csv("NEW decreasing below zero and flat above zero.csv")
D7.to_csv("NEW inANDdecr_below_zero_and_flat_above_zero.csv")
D8.to_csv("NEW flat_below_zero_and_mon_decr_above_zero.csv")
D9.to_csv("NEW flat_below_zero_and_mon_incr_above_zero.csv")
D10.to_csv("NEW flat_below_zero_and_inANDdecr_above_zero.csv")
D11.to_csv("NEW flat_below_zero_and_decr_and_stable_above_zero.csv")
D12.to_csv("NEW flat_below_zero_and_incr_and_stable_above_zero.csv")
D13.to_csv("NEW flat_below_zero_and_inANDdecr_and_stable_above_zero.csv")
D14.to_csv("NEW decreasing and stable below zero and flat above zero.csv")
D15.to_csv("NEW increasing and stable below zero and flat above zero.csv")
D16.to_csv("NEW increasing below zero and decreasing above zero.csv")
D17.to_csv("NEW decreasing below zero and increasing above zero.csv")
D18.to_csv("NEW decreasing below zero and decreasing and stable above zero.csv")
D19.to_csv("NEW increasing below zero and decreasing and stable above zero.csv")
D20.to_csv("NEW decreasing below zero and increasing and stable above zero.csv")
D21.to_csv("NEW decreasing below zero and inANDdecr and stable above zero.csv")
D22.to_csv("NEW inANDdecr_below_zero_and_decr_and_stable_above_zero.csv")
D23.to_csv("NEW increasing and stable below zero and decreasing above zero.csv")
D24.to_csv("NEW decreasing and stable below zero and decreasing and stable above zero.csv")
D25.to_csv("NEW inANDdecr_and_stable_below_zero_and_decreasing_above_zero.csv")
D27.to_csv("NEW decreasing above zero.csv")
D28.to_csv("NEW one to three data point above zero.csv")
D29.to_csv("NEW increasing above zero.csv")
D30.to_csv("NEW inANDdecr_above_zero.csv")
D32.to_csv("NEW decreasing below zero.csv")
D33.to_csv("NEW increasing below zero.csv")
D34.to_csv("NEW one to three data point below zero.csv")
D35.to_csv("NEW inANDdecr_below_zero.csv")
D37.to_csv("NEW decreasing below zero and decreasing above zero.csv")
D38.to_csv("NEW one_to_three_data_point_below_zero_and_mon_decr_above_zero.csv")
D39.to_csv("NEW inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero.csv")
D40.to_csv("NEW decreasing and stable below zero and inANDdecr above zero.csv")
D41.to_csv("NEW increasing and stable below zero and inANDdecr above zero.csv")
D42.to_csv("NEW increasing and stable below zero.csv")
D43.to_csv("NEW decreasing and stable below zero.csv")
D44.to_csv("NEW decreasing and stable above zero.csv")
D45.to_csv("NEW increasing and stable above zero.csv")
D46.to_csv("NEW inANDdecr_and_stable_below_zero.csv")
D47.to_csv("NEW inANDdecr_and_stable_above_zero.csv")
D48.to_csv("NEW decreasing and stable below zero and one to three data point above zero.csv")
D49.to_csv("NEW decreasing below zero and one to three data point above zero.csv")
D50.to_csv("NEW inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero.csv")
D51.to_csv("NEW increasing and stable below zero and one to three data point above zero.csv")
D52.to_csv("NEW inANDdecr_below_zero_and_one_to_three_data_point_above_zero.csv")
D53.to_csv("NEW increasing below zero and one to three data point above zero.csv")
D54.to_csv("NEW decreasing below zero and one to three data point above zero.csv")
D55.to_csv("NEW one_to_three_data_point_below_zero_and_flat_above_zero.csv")
D56.to_csv("NEW flat_below_zero_and_one_to_three_data_point_above_zero.csv")


In [1097]:
Dtotal= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
Dtotal= Dtotal.append(pd.DataFrame(flat_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(flat_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_decr_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_incr_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_decr_and_stable_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(mon_incr_and_stable_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(F_below_zero_and_F_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(decreasing_and_stable_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(flat_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decreasing_and_stable_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(flat_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_below_zero_and_flat_above_zero))   
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_increasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_flat_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_inANDdecr_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(incr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(decr_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(incr_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_below_zero))
Dtotal= Dtotal.append(pd.DataFrame(decr_both_positive_and_negative))
Dtotal= Dtotal.append(pd.DataFrame(incr_both_positive_and_negative))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_incr_below_zero_and_inANDdecr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_incr_below_zero_and_decr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_inANDdecr_below_zero_and_decr_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(inANDdecr_and_stable_below_zero_and_decreasing_above_zero))
Dtotal= Dtotal.append(pd.DataFrame(one_to_three_data_point_below_zero_and_decreasing_above_zero))
              
Dtotal.to_csv("NEW all reactions -30 to 30 lactose.csv")


In [260]:
one_at_zero
flat_above_zero
flat_below_zero
one_to_three_data_point_both_positive_and_negative
flat_below_zero_and_flat_above_zero
mon_decr_and_stable_below_zero
mon_incr_and_stable_below_zero
mon_decr_and_stable_above_zero
mon_incr_and_stable_above_zero
inANDdecr_and_stable_below_zero
inANDdecr_and_stable_above_zero

mon_decr_below_zero_and_flat_above_zero
mon_incr_below_zero_and_flat_above_zero
inANDdecr_below_zero_and_flat_above_zero
flat_below_zero_and_diff_indecr_above_zero
flat_below_zero_and_mon_decr_above_zero
flat_below_zero_and_mon_incr_above_zero
flat_below_zero_and_inANDdecr_above_zero

flat_below_zero_and_decr_and_stable_above_zero
flat_below_zero_and_incr_and_stable_above_zero
flat_below_zero_and_inANDdecr_and_stable_above_zero
incr_and_stable_below_zero_and_flat_above_zero
decr_and_stable_below_zero_and_flat_above_zero
decr_both_positive_and_negative
incr_both_positive_and_negative
inANDdecr_both_positive_and_negative

incr_below_zero_and_decr_and_stable_above_zero
decr_below_zero_and_decr_and_stable_above_zero
incr_below_zero_and_incr_and_stable_above_zero
incr_below_zero_and_inANDdecr_and_stable_above_zero
inANDdecr_below_zero_and_decr_and_stable_above_zero
decreasing_and_stable_below_zero_and_decreasing_above_zero
increasing_and_stable_below_zero_and_decreasing_and_stable_above_zero
inANDdecr_and_stable_below_zero_and_decreasing_above_zero

decr_and_stable_below_zero_and_mon_decr_above_zero
decr_above_zero
one_to_three_data_point_above_zero
inANDdecr_above_zero
decr_below_zero
incr_below_zero
one_to_three_data_point_below_zero
inANDdecr_below_zero

inANDdecr_incr_below_zero_and_decr_above_zero
one_to_three_data_point_below_zero_and_mon_decr_above_zero
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero
incr_and_stable_below_zero_and_inANDdecr_above_zero
decr_and_stable_below_zero_and_inANDdecr_above_zero


['PPA', 'PIuabcpp', 'MNt2pp']

In [ ]:
flat_above_zero= list(flat_above_zero) + list(flat3_above_zero)
flat_above_zero